# บทที่ 2 - ชุดข้อมูลมหัศจรรย์และถิ่นที่อยู่

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ai-builders/curriculum/blob/main/notebooks/02_fantastic_datasets.ipynb)

ในปัจจุบันชุดข้อมูลที่มีพร้อมทั้งปริมาณและคุณภาพเป็นส่วนสำคัญในการสร้าง ML models ในบทเรียนนี้เราจะเรียนรู้วิธีการหาข้อมูลมาเทรนโมเดลของเราทั้งจากชุดข้อมูล open data, web scraping, หรือสร้างขึ้นมาเองจากโมเดลและโค้ด open source ทั้งนี้การหาข้อมูลมาเทรนโมเดลจากแหล่งข้อมูลสาธารณะที่กล่าวมานั้นเราต้องให้ความสำคัญเรื่องลิขสิทธิ์และจริยธรรม (แม้แต่โมเดลเองก็สร้างข้อมูลที่ผิดลิขสิทธิ์-จริยธรรมได้; เรียนเพิ่มเติมในบทที่ 7)

## ติดตั้ง Package ที่ต้องใช้งาน

In [1]:
#fastai
!pip install -q fastbook==0.0.29

#ctgan
!pip install -q sdv==0.18.0

#review generation
!pip install -q transformers==4.27.3

#DuckDuckGo Search API
!pip install -q duckduckgo_search==5.3.0b4

## Data-centric AI เมื่อปริมาณ-คุณภาพข้อมูลสำคัญเท่ากับหรือมากกว่าคุณภาพโมเดล

ในปัจจุบันเรามีสถาปัตยกรรมประสิทธิภาพสูงมากมายให้เลือกใช้ทำ ML models (images - ResNet/EfficientNet, texts - BERT family, tabular data - gradient boosted trees) ปริมาณและคุณภาพของชุดข้อมูลจึงเริ่มมีความสำคัญมากยิ่งขึ้นในการทำให้ประสิทธิภาพของโมเดลของเราดีขึ้น

แนวคิด Data-centric AI ถูกทำให้เป็นที่รู้จักโดย [Andrew Ng และ Landing AI](https://landing.ai/data-centric-ai/) กล่าวคือแทนที่เราจะพยามสร้างโมเดลที่มีประสิทธิภาพยิ่งขึ้น (และโดยส่วนใหญ่แล้วซับซ้อน ใหญ่ และใช้งานยากขึ้น)ให้เรามาใช้เวลาในการ 1) เพิ่มปริมาณข้อมูลและ 2) พัฒนาคุณภาพข้อมูลแทน

ยกตัวอย่าง Data-centric AI ในการตัดคำภาษาไทยด้วยโมเดลโดย [@kornwtp](https://medium.com/@kornwtp/) ในบทความ [Data-Centric ML for Thai Word Segmentation](https://medium.com/@kornwtp/data-centric-for-thai-word-segmentation-e31a79db650d) พวกเขาทำการทดลองตัดคำภาษาไทยด้วยชุดข้อมูล [VISTEC-TP-TH-2021](https://github.com/mrpeerat/OSKut/tree/main/VISTEC-TP-TH-2021) (ประมาณ 5 หมื่นประโยค) ด้วยโมเดล [deepcut](https://github.com/rkcosmos/deepcut) อย่างที่เห็นในตารางรายงานผล การเพิ่มขนาดโมเดลถึง 1 เท่าตัวมีผลให้ความแม่นยำเพิ่มขึ้นเพียงแค่ 1.37% กลับกันการเพิ่มข้อมูลเพียงแค่ 1,500 ตัวอย่างทำให้ความแม่นยำเพิ่มขึ้นถึง 4.90% ยิ่งถ้าข้อมูลที่เพิ่มถูกทำความสะอาดแล้วจะทำให้เพิ่มขึ้นถึง 7.48% (แน่นอนว่าถ้ายิ่งโมเดลใหญ่และข้อมูลเยอะก็ยิ่งดี)

<img src=https://miro.medium.com/max/1400/0*J2vA5iRnIMfuDDAc width="500px">

## ชุดข้อมูลสาธารณะ - Open Data and Open Source

ชุดข้อมูล open data และโค้ด open source ถือเป็นสาธารณูปโภคขั้นพื้นฐานของการทำ ML และโชคดีที่น่าจะเป็นสาธารณูปโภคขั้นพื้นฐานไม่กี่อย่างที่เราในฐานะบุคคลสามารถช่วยกันสร้างได้ด้วยการนำไปใช้-เผยแพร่ชุดข้อมูลและโค้ด

โดยทั่วไปแล้ว ชุดข้อมูลจะถูกเผยแพร่ภายใต้ [Creative Commons Licenses](https://creativecommons.org/) ที่นิยมดังต่อไปนี้

* `CC` - Creative Commons
* `BY` (Attribution) - ใช้ทำซ้ำ แจกจ่าย ดัดแปลงได้ แต่ต้องอ้างอิงเจ้าของผลงาน
* `SA` (ShareAlike) - ใช้ทำซ้ำ แจกจ่าย ดัดแปลงได้ แต่ต้องคง License เดิมไว้
* `NC` (NonCommercial) - ใช้ทำซ้ำ แจกจ่าย ดัดแปลงได้ แต่ห้ามใช้เพื่อการค้า
* `ND` (NoDerivatives) - ใช้ทำซ้ำ แจกจ่ายได้ แต่ห้ามดัดแปลง

เราแนะนำให้คุณแบ่งปันผลงานเป็น `CC-BY-SA` เพื่อให้ชุมชนนักพัฒนาสามารถนำไปต่อยอดได้ เหมือนเช่นที่เรานำงานของชุมชนมาต่อยอด

คุณสามารถเลือก License ที่คุณต้องการได้[ที่นี่](https://creativecommons.org/choose/)

### แหล่งค้นหาชุดข้อมูล

ต่อไปนี้คือแหล่งข้อมูลหลักๆที่คุณสามารถพบเจอชุดข้อมูล open data

แหล่งค้นหาชุดข้อมูลทั่วไป
* [Kaggle](https://www.kaggle.com/datasets)
* [Google Dataset Search](https://datasetsearch.research.google.com/)
* [Papers with Code](https://paperswithcode.com/datasets)
* [Tensorflow Datasets](https://www.tensorflow.org/datasets)

แหล่งค้นหาชุดข้อมูล Images
* [Open Images Dataset](https://storage.googleapis.com/openimages/web/index.html)
* [torchvision.datasets](https://pytorch.org/vision/stable/datasets.html)

แหล่งค้นหาชุดข้อมูล NLP และ Speech
* [Hugging Face Datasets](https://huggingface.co/datasets)
* [torchtext.datasets](https://pytorch.org/text/stable/datasets.html)
* [torchaudio.datasets](https://pytorch.org/audio/stable/datasets.html)
* [NLP for Thai](https://nlpforthai.com/)


แหล่งค้นหาชุดข้อมูล Tabular Data
* [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/index.php)

### ชื่อ Task ไว้ใช้ค้นหาชุดข้อมูล

อีกแนวทางการค้นหาข้อมูลคือค้นหา "[ชื่อ task] datasets" ใน search engine

Images
* Image classification; จำแนกรูปภาพ
* Object detection; จับวัตถุในรูปภาพ
* Image segmentation; จำแนก pixel ในรูปภาพ
* Image information retrieval (search); ค้นหารูปภาพด้วยรูปภาพ
* Image captioning; ใส่คำบรรยายให้รูปภาพ
* Visual question answering; ตอบคำถามจากรูปภาพ
* Image generation; สร้างรูปภาพ
* Image reconstruction; ซ่อมแซมรูปภาพ (รูปเก่า, เซนเซอร์ ฯลฯ)
* Style transfer; เปลี่ยนสไตล์ของรูปภาพ

NLP
* Sequence classification; จำแนกข้อความ
* Token classification; จำแนกหน่วยคำในข้อความ
* Extractive question answering; ตอบคำถามจากบทความ
* Machine translation; แปลภาษา
* Summarization; ย่อความ
* Paraphrasing; ถอดความ
* Optical character recognition; เปลี่ยนตัวอักษรในภาพเป็นข้อความ
* Text information retrieval (search); ค้นหาข้อความด้วยข้อความ
* Text generation; สร้างข้อความ
* Dialogue; สร้างบทสนทนาตอบโต้

Speech
* Speech classification; จำแนกเสียง
* Automatic speech recognition; ถอดเสียงเป็นข้อความ
* Speech synthesis; สร้างเสียง (จากข้อความ)
* Speaker recognition; จำแนกผู้พูด

Tabular Data
* Multi-class classification; จำแนกตัวอย่าง (1 ตัวอย่าง = 1 ประเภท)
* Multi-label classification; จำแนกตัวอย่าง (1 ตัวอย่าง = มากกว่า 1 ประเภท)
* Regression; ทำนายจำนวนจริง
* Time series forecasting; ทำนายตัวเลขในเชิงเวลา
* Recommendation; แนะนำสิ่งของ

## หารูปภาพจาก DuckDuckGo Image Search API

เทคนิคแนะนำโดย [fastai v4 part1 lesson 2](https://github.com/fastai/fastbook/blob/master/02_production.ipynb) ผู้ที่สนใจสามารถไปติดตามบทเรียนต้นทางได้ที่ [course.fast.ai](https://course.fast.ai/) สามารถใช้ search engine อื่นๆ เช่น Google, Bing เป็นต้น เราเลือกใช้ DuckDuckGo เนื่องจากความสะดวกที่สามารถส่งคำขอค้นหารูปภาพได้โดยไม่ต้องใช้ API key

In [2]:
from fastbook import *
from fastai.vision.widgets import *
from tqdm.auto import tqdm
from duckduckgo_search import DDGS

#download from duckduckgo
def search_images_ddg(key, max_n=200):
     """Search for 'key' with DuckDuckGo and return a unique urls of 'max_n' images
        (Adopted from https://github.com/deepanprabhu/duckduckgo-images-api)
     """
     url        = 'https://duckduckgo.com/'
     params     = {'q':key}
     res        = requests.post(url,data=params)
     searchObj  = re.search(r'vqd=([\d-]+)\&',res.text)
     if not searchObj: print('Token Parsing Failed !'); return
     requestUrl = url + 'i.js'
     headers    = {'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:71.0) Gecko/20100101 Firefox/71.0'}
     params     = (('l','us-en'),('o','json'),('q',key),('vqd',searchObj.group(1)),('f',',,,'),('p','1'),('v7exp','a'))
     urls       = []
     while True:
         try:
             res  = requests.get(requestUrl,headers=headers,params=params)
             data = json.loads(res.text)
             for obj in data['results']:
                 urls.append(obj['image'])
                 max_n = max_n - 1
                 if max_n < 1: return L(set(urls))     # dedupe
             if 'next' not in data: return L(set(urls))
             requestUrl = url + data['next']
         except:
             pass

def search_images_ddg_v2(key,max_n=200):
    """Search for 'key' with DuckDuckGo and return a unique urls of 'max_n' images
        Note: Instead of using our own implementation, we using library instead!
    """
    responses = DDGS().images(
                          keywords=key,
                          region="wt-wt",
                          safesearch="off",
                          size=None,
                          color=None,
                          type_image=None,
                          layout=None,
                          license_image=None,
                          max_results=max_n,
                      )
    urls = [ r['image'] for r in responses ]
    return urls

In [3]:
#save ไว้ที่ไหน
path = Path('images')
#เอารูปอะไรบ้าง (labels)
labels = ['chocolate chip cookies','raisin cookies']
#เอา label ละกี่รูป
MAX_N = 100

if not path.exists():
    path.mkdir()
for l in tqdm(labels):
    dest = (path/l)
    dest.mkdir(exist_ok=True)
    results = search_images_ddg_v2(l, max_n=MAX_N)
    download_images(dest, urls=results)

  0%|          | 0/2 [00:00<?, ?it/s]

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


In [4]:
#เช็คว่าได้รูปอะไรบ้าง
fns = get_image_files(path)
fns

(#289) [Path('images/raisin cookies/cc228d53-8c4c-4322-b846-65a2174aae09.jpg'),Path('images/raisin cookies/732a2f1e-322c-414a-8f74-fa4c90dff5f5.jpg'),Path('images/raisin cookies/6f120f4d-876f-4252-be56-659a04ab93af.jpg'),Path('images/raisin cookies/5d65ee2f-9f31-414b-abe9-5a2245b9fa0a.jpg'),Path('images/raisin cookies/d74a0d7a-357f-4d34-975d-b81f0ae02224.jpg'),Path('images/raisin cookies/4cedcd1c-3633-42ce-bab6-9689ac9ab02b.jpg'),Path('images/raisin cookies/5f4bc315-7da3-4211-b1d6-ec7f246a01b1.jpg'),Path('images/raisin cookies/3e1d3a1e-0ad4-454f-ad27-569a65752d52.jpg'),Path('images/raisin cookies/9256a407-f47b-45e4-b8bf-1239583159da.jpg'),Path('images/raisin cookies/478f795d-80c4-492f-9085-42e8c1e6fa76.jpg')...]

In [5]:
#ดูว่าหารูปไหนไม่เจอบ้าง
failed = verify_images(fns)
failed

(#2) [Path('images/raisin cookies/4cedcd1c-3633-42ce-bab6-9689ac9ab02b.jpg'),Path('images/raisin cookies/087e9bd6-452d-4c4c-9751-d233917c5384.jpg')]

In [6]:
#นำรูปที่หาไม่เจอออก
failed.map(Path.unlink)

(#2) [None,None]

## Web Scraping ดึงข้อมูลจากเว็บไซต์สาธารณะ

Web Scraping คือการเขียนโปรแกรมดึงข้อมูลจากเว็บไซต์สาธารณะ ข้อควรระวังคือแม้โดยความเป็นจริงแล้วข้อมูลเหล่านั้นจะถูกเปิดเผยให้ใครเข้าไปดูก็ได้ แต่ไม่ได้หมายความว่าเจ้าของเว็บไซต์จะอยากให้เราดึงข้อมูลจำนวนมากจากเว็บไซต์ของพวกเขา ไม่ว่าจะด้วยเหตุผลด้านลิขสิทธิ์การใช้งานหรือการเพิ่มภาระให้เซิร์ฟเวอร์ของเว็บไซต์นั้นๆ ยกตัวอย่างเช่น Facebook หรือ Twitter ที่ข้อตกลงการใช้งานของพวกเขาไม่อนุญาตให้เราดึงข้อมูลด้วย Web Scraping แต่อนุญาตให้ดึงข้อมูลผ่าน API อย่างเป็นทางการแทน

หากไม่แน่ใจในนโยบายการใช้งาน ทางที่ดีควรติดต่อขออนุญาตจากเจ้าของเว็บไซต์ก่อน นี่คือตัวอย่างที่ทางทีมงานติดต่อไปยัง [soccersuck.com](https://www.soccersuck.com) เพื่อขออนุญาตนำมาใช้สอน

> ทีมงาน AI Builders: สวัสดีครับ ผมทำโครงการสอนเด็กมัธยมทำ AI ชื่อ AI Builders (https://www.facebook.com/aibuildersx) อยู่ครับ พอดีครั้งนี้เราจะจัดสอนช่วงหลังสงกรานต์ และมีหัวข้อหนึ่งคือ web scraping หรือการเขียนโปรแกรมดึงเนื้อหาจากเว็บไซต์ อยากขออนุญาตใช้ https://www.soccersuck.com/boards/oldnews เป็นตัวอย่างในการฝึกสอนครับ โดนสอนให้น้องๆเขียนโปรแกรมดึงพาดหัวข่าว, url ข่าว, และเนื้อข่าวเฉพาะกระทู้ที่อยู่ใน old news 3 หน้าเท่านั้น โปรแกรมทั้งหมดใช้เพื่อสาธิตในการศึกษาครับ นักเรียนมีจำนวน 50 คน คิดว่าไม่มีปัญหา traffic แม้จะกดรันโค้ดพร้อมกันทุกคนครับ

> Soccersuck: ได้ครับไม่มีปัญหา



### Static Web

สมมุติว่าเราอยากทำโมเดล "เขียนพาดหัวข่าวฟุตบอลจากเนื้อข่าวในเว็บ [soccersuck.com](https://www.soccersuck.com/boards/oldnews)"

In [7]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

#### ดึง HTML มาเปลี่ยนเป็น Soup

In [8]:
#package สำหรับจัดการ html เรียกว่า BeautifulSoup
def get_soup(url):
    with requests.get(url) as r:
        soup = BeautifulSoup(r.text, features='html.parser')
    return soup

In [9]:
#ดึงข้อมูล html จากเว็บไซต์มาเปลี่ยนเป็น soup
url = f'https://www.soccersuck.com/boards/oldnews/1/'
soup = get_soup(url)
soup

﻿<!DOCTYPE HTML>

<html xmlns="http://www.w3.org/1999/xhtml" xmlns:fb="https://www.facebook.com/2008/fbml" xmlns:og="http://opengraphprotocol.org/schema/">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/> <title>
		#### ได้รู้ความจริง ได้ยิ่งกว่าฟุตบอล อ่าน soccersuck.com####	</title>
<link href="/img/web/ss_logo_f_16.ico" rel="shortcut icon"/>
<meta content="app-id=1042191005" name="apple-itunes-app"/>
<meta content="app-id=com.soccersuck.devtab.com.soccersuck" name="google-play-app"/>
<link href="https://www.soccersuck.com/css/jquery.smartbanner.css" media="screen" rel="stylesheet" type="text/css"/>
<meta content="ข่าวฟุตบอล ข่าวบอล นักฟุตบอล ผลการแข่งขัน ข่าวลือ ข่าวการย้ายทีม บอลอังกฤษ บอลเยอรมัน บอลอิตาลี บอลสเปน ฟุตบอล บอบทีมชาติ บอลไทย บอลทีมชติไทย เว็บบอร์ด วาไรตี้ ภาพเซ็กซี่ 18+ โปรแกรมการแข่งขัน โปรแกรมถ่ายทอดสด" name="description">
<meta content="ข่าวฟุตบอล,ข่าวบอล,ผลการแข่งขัน,นักฟุตบอล,บอลอังกฤษ,บอลเยอรมัน,บอลอิตาลี,บอลสเปน,ฟุตบอล,ข่าวการย้ายทีม,

#### หา Tag ต่างๆใน Soup

In [10]:
#หา tag ชื่อ div ที่มี class oldnew_p_tr
#เราสามาร?ถหาโดย id ได้ด้วย แค่เปลี่ยนเป็น id='id_you_are_looking_for'
soup.find('div', class_='oldnew_p_tr')

<div class="oldnew_p_tr" style="background: #C4E29B;">
<img alt="" src="https://www.soccersuck.com/images/ucl.jpg" style="margin: 0px 5px -1px 2px;width: 20px;"/> <a href="https://www.soccersuck.com/boards/topic/2431838" target="_blank">
                    ชาบี้มั่นใจ บาร์ซ่าบู๊ PSG เร้าใจไม่แพ้นัดแรก                    </a>
</div>

In [11]:
soup.find('div', class_='oldnew_p_tr').find('a')

<a href="https://www.soccersuck.com/boards/topic/2431838" target="_blank">
                    ชาบี้มั่นใจ บาร์ซ่าบู๊ PSG เร้าใจไม่แพ้นัดแรก                    </a>

In [12]:
#เอา text จาก tag <a>
soup.find('div', class_='oldnew_p_tr').find('a').text.strip(),\
#เอา href จาก tag <a>
soup.find('div', class_='oldnew_p_tr').find('a').get('href')

'https://www.soccersuck.com/boards/topic/2431838'

In [13]:
df = pd.DataFrame([
{'headline_abridged': i.text.strip(),
 'url': i.find('a').get('href')} \
 #find() จะหาแค่ tag แรกที่เจอ แต่ find_all() จะหาทุก tag ที่ตรงเงื่อนไข
 for i in soup.find_all('div', class_='oldnew_p_tr')]).head(10)
df

,headline_abridged,url
0,ชาบี้มั่นใจ บาร์ซ่าบู๊ PSG เร้าใจไม่แพ้นัดแรก,https://www.soccersuck.com/boards/topic/2431838
1,"สิงห์ใช้ออพชั่นขยายสัญญา 'เอนโซ, มูดรีค' อยู่ยาวยั...",https://www.soccersuck.com/boards/topic/2431832
2,เผย ETH เรียก 'การ์นาโช่' คุยส่วนตัวเคลียร์ดราม่าก...,https://www.soccersuck.com/boards/topic/2431825
3,แชมป์เปี้ยนชิพเลือก 'แม็คเคนน่า' กุนซือแห่งปี,https://www.soccersuck.com/boards/topic/2431800
4,ฝันเป็นจริง! แฟนคลับนิวชาวไทยถูกเชิญดูบอลถึง เซนต์ เจมส์ พาร...,https://www.soccersuck.com/boards/topic/2431798
5,เตรียมตัวมา 2 ปี! ‘โค้ชหระ’ มอง U23 ทุกทีมมีโอกาสออกได้ทั้งส...,https://www.soccersuck.com/boards/topic/2431793
6,เช็คความพร้อม! ‘อิสสระ-อิชิอิ’ ตรวจสนาม ไทย บู๊ อิรัก ประเดิ...,https://www.soccersuck.com/boards/topic/2431751
7,กิจวัตรใหม่! ขุนแจ็ค' ควงน้องซาช่าซาวน่าทุกคืน,https://www.soccersuck.com/boards/topic/2431747
8,หงส์จ้องอยู่! พี่เสือนัดตัวแทน 'ซาเน่' ถกสัญญาใหม่...,https://www.soccersuck.com/boards/topic/2431744
9,เจ็บเล็กน้อย...ผีปัด 'เดอะ หมุน' เกาเหลา ETH,https://www.soccersuck.com/boards/topic/2431738


#### ดึงข้อมูลจาก url ย่อย

In [14]:
#จากหน้า list　ข่าว พาดหัวจะแสดงได้ไม่เต็ม
df.loc[0]

headline_abridged      ชาบี้มั่นใจ บาร์ซ่าบู๊ PSG เร้าใจไม่แพ้นัดแรก
url                  https://www.soccersuck.com/boards/topic/2431838
Name: 0, dtype: object

In [15]:
#เราสามารถเข้าไปใน url ของข่าวแต่ละข่าวเพื่อไปเอาพาดหัวแบบเต็มและเนื้อข่าวได้
soup = get_soup(df.loc[0,'url'])
soup.find('div', class_='post_head').text.strip(),\
soup.find('div', class_='post_desc').text.strip()

('ชาบี้มั่นใจ บาร์ซ่าบู๊ PSG เร้าใจไม่แพ้นัดแรก',
 'ชาบี้เอร์นานเดซ บอสใหญ่บาร์เซโลน่า เชื่อว่า เกมปะทะปารีส แซงต์ แชร์กแมง ในยูฟ่า แชมเปี้ยนส์ลีก รอบก่อนรองชนะเลิศ นัดสอง จะต้องเร้าใจไม่แพ้เกมแรกที่พวกเขาบุกไปปราบทีมแชมป์ลีกเอิงถึงถิ่น 3-2\n\nทีมของ ชาบี้ สร้างผลงานยอดเยี่ยมบุกไปยัดเยียดความปราชัยให้ เปแอสเช ได้ถึงฝรั่งเศส 3-2 เมื่อสัปดาห์ที่แล้ว  สถานการณ์ได้เปรียบที่จะตบเท้าเข้ารอบรองชนะเลิศได้เป็นครั้งแรกนับตั้งแต่ปี 2019\n\n"เราไม่ใช่ทีมที่นำแล้วอุด เราต้องการแย่งบอลจากเปแอสเชและเป็นผู้ชนะในเกมนี้" ชาบี้ กล่าวในเพรส คอนเฟอเรนซ์ ก่อนเกมสำคัญวันอังคารนี้\n\n"ผมหวังว่าเราจะอยู่ในจุดที่ดีที่สุดของตัวเอง พวกเขาจะทดสอบขีดจำกัดของเรา ผมคิดว่าเกมนี้จะคล้ายกับนัดแรก"\n\n"มันคือแชมเปี้ยนส์ลีก นี่คือทีมปารีส และเป็นเกมที่จะไม่มีใครถอยลงไปตั้งรับ"\n\n""หลุยส์ เอ็นริเก้ จะไม่ตั้งรับ เขาจะมาที่นี่เพื่อเล่นเกมเพรสสูง เราต้องโชว์บุคลิกให้มากเพื่อผ่านเข้ารอบรองชนะเลิศ"')

In [16]:
#เขียนฟังชั่นเพื่อเข้าไปดึงข้อมูลจากหน้าข่าว
def get_post_head_desc(url):
    soup = get_soup(url)
    return {'post_head': soup.find('div', class_='post_head').text.strip(),
            'post_desc': soup.find('div', class_='post_desc').text.strip()}

In [17]:
#วนลูป (map) เพื่อไปดึงพาดหัวแบบเต็มและเนื้อข่าว
df2 = pd.DataFrame(df.url.map(get_post_head_desc).tolist())
df = pd.concat([df,df2],1)
df

<ipython-input-17-3d26f9c8868d>:3: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  df = pd.concat([df,df2],1)


,headline_abridged,url,post_head,post_desc
0,ชาบี้มั่นใจ บาร์ซ่าบู๊ PSG เร้าใจไม่แพ้นัดแรก,https://www.soccersuck.com/boards/topic/2431838,ชาบี้มั่นใจ บาร์ซ่าบู๊ PSG เร้าใจไม่แพ้นัดแรก,"ชาบี้เอร์นานเดซ บอสใหญ่บาร์เซโลน่า เชื่อว่า เกมปะทะปารีส แซงต์ แชร์กแมง ในยูฟ่า แชมเปี้ยนส์ลีก รอบก่อนรองชนะเลิศ นัดสอง จะต้องเร้าใจไม่แพ้เกมแรกที่พวกเขาบุกไปปราบทีมแชมป์ลีกเอิงถึงถิ่น 3-2\n\nทีมของ ชาบี้ สร้างผลงานยอดเยี่ยมบุกไปยัดเยียดความปราชัยให้ เปแอสเช ได้ถึงฝรั่งเศส 3-2 เมื่อสัปดาห์ที่แล้ว สถานการณ์ได้เปรียบที่จะตบเท้าเข้ารอบรองชนะเลิศได้เป็นครั้งแรกนับตั้งแต่ปี 2019\n\n""เราไม่ใช่ทีมที่นำแล้วอุด เราต้องการแย่งบอลจากเปแอสเชและเป็นผู้ชนะในเกมนี้"" ชาบี้ กล่าวในเพรส คอนเฟอเรนซ์ ก่อนเกมสำคัญวันอังคารนี้\n\n""ผมหวังว่าเราจะอยู่ในจุดที่ดีที่สุดของตัวเอง พวกเขาจะทดสอบขีดจำกัดของเรา ผมคิดว่า..."
1,"สิงห์ใช้ออพชั่นขยายสัญญา 'เอนโซ, มูดรีค' อยู่ยาวยั...",https://www.soccersuck.com/boards/topic/2431832,"สิงห์ใช้ออพชั่นขยายสัญญา 'เอนโซ, มูดรีค' อยู่ยาวยัน 30","รายงานข่าวจาก The Standard เปิดเผยว่า ""สิงห์บลูส์"" เชลซี เปิดใช้ออพชั่นขยายสัญญาระยะยาวให้กับ เอนโซ แฟร์นานเดซ และ มิคายโล มูดรีค ยาวไปจนถึงอายุ 30 ปี\n\n\nเชลซี เซ็นสัญญากับ เอนโซ ด้วยค่าตัวที่แพงเป็นสถิติอังกฤษที่ 107 ล้านปอนด์ จากเบนฟิก้า เมื่อมกราคม ปี 2023 ขณะที่ มูดรีค ย้ายมาจากชัคตาร์ โดเนสก์ ค่าตัว 88 ล้านปอนด์ในตลาดรอบเดียวกัน\n\nรายงานจาก The Standard อ้างอิงเอกสารของเอฟเอ ที่เปิดเผยว่า เอนโซ เหลือสัญญาอีก 8 ปีกับค่าเหนื่อย 180,000 ปอนด์ต่อสัปดาห์ ซึ่งสัญญาของเขาจะหมดลงในปี 2032 หรือตอนอายุ 31 ปี\n\nขณะที่ มูดรีค เหลือสัญญา 7 ปี กับค่าเหนื่อย 87,000 ปอนด์ต่อสัปดาห์ และสตาร์ยูเครน..."
2,เผย ETH เรียก 'การ์นาโช่' คุยส่วนตัวเคลียร์ดราม่าก...,https://www.soccersuck.com/boards/topic/2431825,เผย ETH เรียก 'การ์นาโช่' คุยส่วนตัวเคลียร์ดราม่ากดไลค์,รายงานข่าวจาก Manchester Evening News เปิดเผยว่า เอริค เทน ฮาก ได้พูดคุยส่วนตัวกับ อเลฮานโดร การ์นาโช่ หลังดาวรุ่งวัย19 ปี ไปกดไลค์ถูกใจทวิตวิจารณ์บอสชาวดัตช์หลังเกมเสมอบอร์นมัธ 2-2 เมื่อคืนวันเสาร์ที่ผ่านมา\n\n\nช่วงพักครึ่งของเกมดังกล่าว เทน ฮาก ตัดสินใจเปลี่ยนตัวเอา อาหมัด ดิยาลโล่ ลงไปแทน การ์นาโช่ และทีมก็ดูดีขึ้นขึ้นเล็กน้อยในครึ่งหลัง\n\nเมื่อถามว่าทำไมเขาถึงถอด การ์นาโช่ ออกตอนพักครึ่ง เทน ฮาก ตอบในทำนองโยนความผิดให้ปีกดาวรุ่งผู้นี้\n\nจากนั้นหลังจบเกม โกลด์บริดจ์ ที่มักจะวิจารณ์ทีมตัวเองอยู่เสมอไม่พลาดที่จะจัดหนักใส่ เทน ฮาก โดยเฉพาะการถอด การ์นาโช่ ออก และปีกวัย 19 ปีไปกดไลค์ทั้ง...
3,แชมป์เปี้ยนชิพเลือก 'แม็คเคนน่า' กุนซือแห่งปี,https://www.soccersuck.com/boards/topic/2431800,แชมป์เปี้ยนชิพเลือก 'แม็คเคนน่า' กุนซือแห่งปี,"เคียแรน แม็คเคนน่า อดีตผู้ช่วย แมนเชสเตอร์ ยูไนเต็ด ได้รับเลือกเป็นกุนซือยอดเยี่ยมประจำซีซั่น แชมป์เปี้ยนชิพ จากผลงานคุม อิปสวิช ทาวน์ รั้งจ่าฝูง ขณะที่รางวัลฝั่งนักเตะ ตกเป็นของ คริสเซนซิโอ้ ซัมเมอร์วิลล์ แนวรุก ลีดส์ ยูไนเต็ด\n\nแม็คเคนน่า นั่งแท่นผู้จัดการ ""ม้าขาว"" แบบเต็มตัวเป็นฤดูกาลที่สอง และกำลังทำผลงานนำจ่าฝูงเหนือ เลสเตอร์ ซิตี้ 1 แต้ม ขณะที่การแข่งเหลือ 3 นัดสุดท้าย ลุ้นเลื่อนชั้นสองซีซั่นติด ภายหลังเพิ่งขึ้นมาจาก ลีก วัน \n\nผลงานดังกล่าวทำให้เฮดโค้ชชาวไอร์แลนด์เหนือ ได้รับรางวัลประจำปีเหนือผู้ท้าชิงอย่าง ดาเนียล ฟาร์เก้ ของ ลีดส์ ยูไนเต็ด และ เลียม โรเซเนียร์ จาก ฮัลล์ ซิตี้\n\..."
4,ฝันเป็นจริง! แฟนคลับนิวชาวไทยถูกเชิญดูบอลถึง เซนต์ เจมส์ พาร...,https://www.soccersuck.com/boards/topic/2431798,ฝันเป็นจริง! แฟนคลับนิวชาวไทยถูกเชิญดูบอลถึง เซนต์ เจมส์ พาร์ค สุดฟิน,กลุ่มแฟนคลับทางการของ นิวคาสเซิล ยูไนเต็ด ประเทศไทย (NUFCTH) บินไปดูพรีเมียร์ลีกของทีมรักแบบติดขอบสนาม เมื่อสุดสัปดาห์ที่ผ่านมา\n\nFUN ชวนแฟน ๆ นิวคาสเซิลจากเมืองไทย มาร่วมชมเกมระหว่าง “สาลิกาดง” กับ ท็อตแนม ฮ็อตสเปอร์ ที่สนาม “เซนต์ เจมส์ พาร์ค” ซึ่งผลการแข่งขันจบลงที่เจ้าบ้านเปิดรังถล่มเอาชนะทีมเยือนไปเละเทะ 4-0\n\nคุณวิน หนึ่งในแฟนคลับที่ได้ไปสัมผัสประสบการณ์ฟุตบอลผู้ดีแบบสด ๆ เปิดใจหลังจบเกมว่า “ดีใจมาก ๆ ครับที่ได้มาดู นิวคาสเซิล ถึงขอบสนามเป็นครั้งแรก เป็นประสบการณ์ที่สุดยอดมาก ๆ ผมยังขนลุกอยู่เลยกับบรรยากาศใน เซนต์ เจมส์ พาร์ค ต้องขอขอบคุณสปอนเซอร์ใจดีอย่าง FUN ที่มอบโอกาสนี้ให้กับพ...
5,เตรียมตัวมา 2 ปี! ‘โค้ชหระ’ มอง U23 ทุกทีมมีโอกาสออกได้ทั้งส...,https://www.soccersuck.com/boards/topic/2431793,เตรียมตัวมา 2 ปี! ‘โค้ชหระ’ มอง U23 ทุกท

### Static Web แบบเร็วด้วย Concurrency

บางครั้งแค่ scape ทีละ url อาจจะไม่ทันใจ เราจึงสามารถ scrape ทีละหลายๆ url "พร้อมๆ" กันได้ด้วย concurrency วิธีมีหลากหลาย แต่เราเลือกใช้วิธีที่แนะนำโดย [Nick Becker, RAPIDS Team at NVIDIA](https://beckernick.github.io/faster-web-scraping-python/)

In [18]:
import concurrent.futures

In [19]:
#url ข่าวที่เราต้องการ scrape
urls = df.url.tolist()
urls[:3]

['https://www.soccersuck.com/boards/topic/2431838',
 'https://www.soccersuck.com/boards/topic/2431832',
 'https://www.soccersuck.com/boards/topic/2431825']

In [20]:
#ฟังชั่นเพื่อเข้าไปดึงข้อมูลจากหน้าข่าวอันเดิม
#เพิ่มเติมคือเราจะบอกให้มันเก็บข้อมูลที่ถูก scrape ไว้ใน list ชื่อ res
def get_post_head_desc_append(url, res):
    soup = get_soup(url)
    res.append({'post_head': soup.find('div', class_='post_head').text.strip(),
            'post_desc': soup.find('div', class_='post_desc').text.strip()})

In [21]:
#เราใช้ 20 threads คือให้ดึงเต็มที่ 20 url　พร้อมๆกัน
MAX_THREADS = 20

def scrape_multithread(urls, scrape_func):
    threads = min(MAX_THREADS, len(urls))
    with concurrent.futures.ThreadPoolExecutor(max_workers=threads) as executor:
        executor.map(scrape_func, urls)

In [22]:
#ทำการ scrape แบบ concurrent
from functools import partial

res = []
scrape_multithread(urls, partial(get_post_head_desc_append, res=res))
pd.DataFrame(res)

,post_head,post_desc
0,"สิงห์ใช้ออพชั่นขยายสัญญา 'เอนโซ, มูดรีค' อยู่ยาวยัน 30","รายงานข่าวจาก The Standard เปิดเผยว่า ""สิงห์บลูส์"" เชลซี เปิดใช้ออพชั่นขยายสัญญาระยะยาวให้กับ เอนโซ แฟร์นานเดซ และ มิคายโล มูดรีค ยาวไปจนถึงอายุ 30 ปี\n\n\nเชลซี เซ็นสัญญากับ เอนโซ ด้วยค่าตัวที่แพงเป็นสถิติอังกฤษที่ 107 ล้านปอนด์ จากเบนฟิก้า เมื่อมกราคม ปี 2023 ขณะที่ มูดรีค ย้ายมาจากชัคตาร์ โดเนสก์ ค่าตัว 88 ล้านปอนด์ในตลาดรอบเดียวกัน\n\nรายงานจาก The Standard อ้างอิงเอกสารของเอฟเอ ที่เปิดเผยว่า เอนโซ เหลือสัญญาอีก 8 ปีกับค่าเหนื่อย 180,000 ปอนด์ต่อสัปดาห์ ซึ่งสัญญาของเขาจะหมดลงในปี 2032 หรือตอนอายุ 31 ปี\n\nขณะที่ มูดรีค เหลือสัญญา 7 ปี กับค่าเหนื่อย 87,000 ปอนด์ต่อสัปดาห์ และสตาร์ยูเครน..."
1,ชาบี้มั่นใจ บาร์ซ่าบู๊ PSG เร้าใจไม่แพ้นัดแรก,"ชาบี้เอร์นานเดซ บอสใหญ่บาร์เซโลน่า เชื่อว่า เกมปะทะปารีส แซงต์ แชร์กแมง ในยูฟ่า แชมเปี้ยนส์ลีก รอบก่อนรองชนะเลิศ นัดสอง จะต้องเร้าใจไม่แพ้เกมแรกที่พวกเขาบุกไปปราบทีมแชมป์ลีกเอิงถึงถิ่น 3-2\n\nทีมของ ชาบี้ สร้างผลงานยอดเยี่ยมบุกไปยัดเยียดความปราชัยให้ เปแอสเช ได้ถึงฝรั่งเศส 3-2 เมื่อสัปดาห์ที่แล้ว สถานการณ์ได้เปรียบที่จะตบเท้าเข้ารอบรองชนะเลิศได้เป็นครั้งแรกนับตั้งแต่ปี 2019\n\n""เราไม่ใช่ทีมที่นำแล้วอุด เราต้องการแย่งบอลจากเปแอสเชและเป็นผู้ชนะในเกมนี้"" ชาบี้ กล่าวในเพรส คอนเฟอเรนซ์ ก่อนเกมสำคัญวันอังคารนี้\n\n""ผมหวังว่าเราจะอยู่ในจุดที่ดีที่สุดของตัวเอง พวกเขาจะทดสอบขีดจำกัดของเรา ผมคิดว่า..."
2,เจ็บเล็กน้อย...ผีปัด 'เดอะ หมุน' เกาเหลา ETH,"ESPN สื่อกีฬาชื่อดัง เปิดเผยว่า แมนเชสเตอร์ ยูไนเต็ด ปฏิเสธกระแสข่าว แอนโธนี่ ปีกเบย์เบลดของแมนเชสเตอร์ ยูไนเต็ด ซดเกาเหลากับ เอริก เทน ฮาก จนไม่มีชื่อในเกมเสมอ บอร์นมัธ 2-2 \n\nแอนโทนี่ ไม่มีชื่อในเกมเยือนวิตาลิตี้ สเตเดี้ยม เมื่อวันเสาร์ที่ผ่านมา โดยมีกระแสข่าวตามมาว่าเขามีปัญหากับโค้ชชาวดัตช์จนโดนหั่นหลุดทีม\n\nอย่างไรก็ดี แหล่งข่าวเผยว่า ""ปีศาจแดง"" ไม่แฮปปี้กับข่าวลือที่ว่า ""เดอะ หมุน"" หายหน้าไปเพราะมีปัญหากับเจ้านายอย่างยิ่ง\n\nรายงานยืนยันว่า แอนโทนี่ ไม่ได้เดินทางลงใต้เพราะมีปัญหาบาดเจ็บระหว่างลงซ้อมเมื่อวันศุกร์ จนซ้อมต่อไม่ไหว\n\nอาการบาดเจ็บดังกล่าวไม่ได้ร้ายแรงอะไรและน่าจะเรียกค..."
3,เตรียมตัวมา 2 ปี! ‘โค้ชหระ’ มอง U23 ทุกทีมมีโอกาสออกได้ทั้งสามหน้า,"อิสสระ ศรีทะโร แถลงความพร้อม ทีมชาติไทย U23 ย้ำเป้าหมายมาเพื่อคว้าตั๋วไปโอลิมปิก แต่โฟกัสเกมต่อเกม มองทุกทีมมีโอกาส ออกได้ทั้งสามหน้า\n\nวันที่ 15 เมษายน 2567 เวลา 17.30 น. ตามเวลาประเทศไทย ณ ห้องแถลงข่าว คอลิฟา อินเตอร์เนชันแนล สเตเดียม สมาพันธ์ฟุตบอลแห่งเอเชีย หรือ เอเอฟซี จัดงานแถลงข่าวก่อนการแข่งขันฟุตบอล AFC U23 Asian Cup QATAR 2024 กลุ่มซี \n\nการแถลงข่าวครั้งนี้ประกอบไปด้วย 4 ชาติในกลุ่ม ซี ประกอบไปด้วย ซาอุดีอาระเบีย, อิรัก, ไทย และ ทาจิกิสถาน โดย ไทยได้ส่ง อิสสระ ศรีทะโร หัวหน้าผู้ฝึกสอน เป็นตัวแทน\n\nโดย อิสสระ ศรีทะโร หัวหน้าผู้ฝึกสอนทีมชาติไทย U23 กล่าวว่า ""ทีมชาติไทยมีการเตรีย..."
4,ฝันเป็นจริง! แฟนคลับนิวชาวไทยถูกเชิญดูบอลถึง เซนต์ เจมส์ พาร์ค สุดฟิน,กลุ่มแฟนคลับทางการของ นิวคาสเซิล ยูไนเต็ด ประเทศไทย (NUFCTH) บินไปดูพรีเมียร์ลีกของทีมรักแบบติดขอบสนาม เมื่อสุดสัปดาห์ที่ผ่านมา\n\nFUN ชวนแฟน ๆ นิวคาสเซิลจากเมืองไทย มาร่วมชมเกมระหว่าง “สาลิกาดง” กับ ท็อตแนม ฮ็อตสเปอร์ ที่สนาม “เซนต์ เจมส์ พาร์ค” ซึ่งผลการแข่งขันจบลงที่เจ้าบ้านเปิดรังถล่มเอาชนะทีมเยือนไปเละเทะ 4-0\n\nคุณวิน หนึ่งในแฟนคลับที่ได้ไปสัมผัสประสบการณ์ฟุตบอลผู้ดีแบบสด ๆ เปิดใจหลังจบเกมว่า “ดีใจมาก ๆ ครับที่ได้มาดู นิวคาสเซิล ถึงขอบสนามเป็นครั้งแรก เป็นประสบการณ์ที่สุดยอดมาก ๆ ผมยังขนลุกอยู่เลยกับบรรยากาศใน เซนต์ เจมส์ พาร์ค ต้องขอขอบคุณสปอนเซอร์ใจดีอย่าง FUN ที่มอบโอกาสนี้ให้กับพ...
5,เช็คความพร้อม! ‘อิสสระ-อิชิอิ’ ตรวจสนาม ไทย บู๊ อิรัก ประเดิม U23,"อิสสระ ศรีทะโร หัวหน้าผู้ฝึกสอนทีมชาติไทย U23 และ มาซาทาดะ อิชิอิ ที่ปรึกษาทีม พร้อมทีมงานสตาฟฟ์โค้ช ตรวจสภาพความพร้อมของ อัล ยานูบ สเตเดียม สนามที่ ทีมชาติไทย U23 จะทำการแข่งขัน AFC U23 Asian Cup 2024 นัดแรกที่จะพบกับ อิรัก ในวันที่ 16 เมษายนนี้ ถ่ายทอดสดทาง ทรูวิชั่นส์ True Sport 2 ช่อง 667\n\nสำหรับสนามดังกล่าว เปิดใช้งาน เมื่อวันที่ 16 พฤษภาคม 2019 ที่ผ่านมา และสามารถจุแฟนบอลได้จำนวน 44.325 ที่นั่ง\n\nนอกจากนี้สนามดังกล่าวยังเคยใช้จัดฟุตบอลรายการสำคัญๆมาแล้วมากมายอาทิ อราเบียน กัล์ฟ คัพ 2019, ฟีฟ่า อาหรับ คัพ 2021 (6 นัด), ฟุตบอลโลก 2022 รอบสุดท้าย (7 นัด) และ เอเชียน คัพ 2023 (6 นัด)\..

จะเห็นว่าใช้เวลาเพียง 2 วินาที เทียบกับการดึงทีละ url ที่ใช้เวลากว่า 15 วินาที

In [23]:
%%time
res = []
scrape_multithread(urls, partial(get_post_head_desc_append, res=res))
pd.DataFrame(res)

CPU times: user 1.8 s, sys: 49.3 ms, total: 1.85 s
Wall time: 2.51 s


,post_head,post_desc
0,เช็คความพร้อม! ‘อิสสระ-อิชิอิ’ ตรวจสนาม ไทย บู๊ อิรัก ประเดิม U23,"อิสสระ ศรีทะโร หัวหน้าผู้ฝึกสอนทีมชาติไทย U23 และ มาซาทาดะ อิชิอิ ที่ปรึกษาทีม พร้อมทีมงานสตาฟฟ์โค้ช ตรวจสภาพความพร้อมของ อัล ยานูบ สเตเดียม สนามที่ ทีมชาติไทย U23 จะทำการแข่งขัน AFC U23 Asian Cup 2024 นัดแรกที่จะพบกับ อิรัก ในวันที่ 16 เมษายนนี้ ถ่ายทอดสดทาง ทรูวิชั่นส์ True Sport 2 ช่อง 667\n\nสำหรับสนามดังกล่าว เปิดใช้งาน เมื่อวันที่ 16 พฤษภาคม 2019 ที่ผ่านมา และสามารถจุแฟนบอลได้จำนวน 44.325 ที่นั่ง\n\nนอกจากนี้สนามดังกล่าวยังเคยใช้จัดฟุตบอลรายการสำคัญๆมาแล้วมากมายอาทิ อราเบียน กัล์ฟ คัพ 2019, ฟีฟ่า อาหรับ คัพ 2021 (6 นัด), ฟุตบอลโลก 2022 รอบสุดท้าย (7 นัด) และ เอเชียน คัพ 2023 (6 นัด)\..."
1,ชาบี้มั่นใจ บาร์ซ่าบู๊ PSG เร้าใจไม่แพ้นัดแรก,"ชาบี้เอร์นานเดซ บอสใหญ่บาร์เซโลน่า เชื่อว่า เกมปะทะปารีส แซงต์ แชร์กแมง ในยูฟ่า แชมเปี้ยนส์ลีก รอบก่อนรองชนะเลิศ นัดสอง จะต้องเร้าใจไม่แพ้เกมแรกที่พวกเขาบุกไปปราบทีมแชมป์ลีกเอิงถึงถิ่น 3-2\n\nทีมของ ชาบี้ สร้างผลงานยอดเยี่ยมบุกไปยัดเยียดความปราชัยให้ เปแอสเช ได้ถึงฝรั่งเศส 3-2 เมื่อสัปดาห์ที่แล้ว สถานการณ์ได้เปรียบที่จะตบเท้าเข้ารอบรองชนะเลิศได้เป็นครั้งแรกนับตั้งแต่ปี 2019\n\n""เราไม่ใช่ทีมที่นำแล้วอุด เราต้องการแย่งบอลจากเปแอสเชและเป็นผู้ชนะในเกมนี้"" ชาบี้ กล่าวในเพรส คอนเฟอเรนซ์ ก่อนเกมสำคัญวันอังคารนี้\n\n""ผมหวังว่าเราจะอยู่ในจุดที่ดีที่สุดของตัวเอง พวกเขาจะทดสอบขีดจำกัดของเรา ผมคิดว่า..."
2,ฝันเป็นจริง! แฟนคลับนิวชาวไทยถูกเชิญดูบอลถึง เซนต์ เจมส์ พาร์ค สุดฟิน,กลุ่มแฟนคลับทางการของ นิวคาสเซิล ยูไนเต็ด ประเทศไทย (NUFCTH) บินไปดูพรีเมียร์ลีกของทีมรักแบบติดขอบสนาม เมื่อสุดสัปดาห์ที่ผ่านมา\n\nFUN ชวนแฟน ๆ นิวคาสเซิลจากเมืองไทย มาร่วมชมเกมระหว่าง “สาลิกาดง” กับ ท็อตแนม ฮ็อตสเปอร์ ที่สนาม “เซนต์ เจมส์ พาร์ค” ซึ่งผลการแข่งขันจบลงที่เจ้าบ้านเปิดรังถล่มเอาชนะทีมเยือนไปเละเทะ 4-0\n\nคุณวิน หนึ่งในแฟนคลับที่ได้ไปสัมผัสประสบการณ์ฟุตบอลผู้ดีแบบสด ๆ เปิดใจหลังจบเกมว่า “ดีใจมาก ๆ ครับที่ได้มาดู นิวคาสเซิล ถึงขอบสนามเป็นครั้งแรก เป็นประสบการณ์ที่สุดยอดมาก ๆ ผมยังขนลุกอยู่เลยกับบรรยากาศใน เซนต์ เจมส์ พาร์ค ต้องขอขอบคุณสปอนเซอร์ใจดีอย่าง FUN ที่มอบโอกาสนี้ให้กับพ...
3,เตรียมตัวมา 2 ปี! ‘โค้ชหระ’ มอง U23 ทุกทีมมีโอกาสออกได้ทั้งสามหน้า,"อิสสระ ศรีทะโร แถลงความพร้อม ทีมชาติไทย U23 ย้ำเป้าหมายมาเพื่อคว้าตั๋วไปโอลิมปิก แต่โฟกัสเกมต่อเกม มองทุกทีมมีโอกาส ออกได้ทั้งสามหน้า\n\nวันที่ 15 เมษายน 2567 เวลา 17.30 น. ตามเวลาประเทศไทย ณ ห้องแถลงข่าว คอลิฟา อินเตอร์เนชันแนล สเตเดียม สมาพันธ์ฟุตบอลแห่งเอเชีย หรือ เอเอฟซี จัดงานแถลงข่าวก่อนการแข่งขันฟุตบอล AFC U23 Asian Cup QATAR 2024 กลุ่มซี \n\nการแถลงข่าวครั้งนี้ประกอบไปด้วย 4 ชาติในกลุ่ม ซี ประกอบไปด้วย ซาอุดีอาระเบีย, อิรัก, ไทย และ ทาจิกิสถาน โดย ไทยได้ส่ง อิสสระ ศรีทะโร หัวหน้าผู้ฝึกสอน เป็นตัวแทน\n\nโดย อิสสระ ศรีทะโร หัวหน้าผู้ฝึกสอนทีมชาติไทย U23 กล่าวว่า ""ทีมชาติไทยมีการเตรีย..."
4,เจ็บเล็กน้อย...ผีปัด 'เดอะ หมุน' เกาเหลา ETH,"ESPN สื่อกีฬาชื่อดัง เปิดเผยว่า แมนเชสเตอร์ ยูไนเต็ด ปฏิเสธกระแสข่าว แอนโธนี่ ปีกเบย์เบลดของแมนเชสเตอร์ ยูไนเต็ด ซดเกาเหลากับ เอริก เทน ฮาก จนไม่มีชื่อในเกมเสมอ บอร์นมัธ 2-2 \n\nแอนโทนี่ ไม่มีชื่อในเกมเยือนวิตาลิตี้ สเตเดี้ยม เมื่อวันเสาร์ที่ผ่านมา โดยมีกระแสข่าวตามมาว่าเขามีปัญหากับโค้ชชาวดัตช์จนโดนหั่นหลุดทีม\n\nอย่างไรก็ดี แหล่งข่าวเผยว่า ""ปีศาจแดง"" ไม่แฮปปี้กับข่าวลือที่ว่า ""เดอะ หมุน"" หายหน้าไปเพราะมีปัญหากับเจ้านายอย่างยิ่ง\n\nรายงานยืนยันว่า แอนโทนี่ ไม่ได้เดินทางลงใต้เพราะมีปัญหาบาดเจ็บระหว่างลงซ้อมเมื่อวันศุกร์ จนซ้อมต่อไม่ไหว\n\nอาการบาดเจ็บดังกล่าวไม่ได้ร้ายแรงอะไรและน่าจะเรียกค..."
5,หงส์จ้องอยู่! พี่เสือนัดตัวแทน 'ซาเน่' ถกสัญญาใหม่วีคนี้,"Kicker นิตยสารเมืองเบียร์ เปิดเผยว่า บาเยิร์น มิวนิค เตรียมพูดคุยกับตัวแทนของ เลรอย ซาเน่ ปีกจอมพริ้วเรื่องการขายสัญญาฉบับใหม่หลังเกมพบ อาร์เซน่อล ในศึกแชมเปี้ยนส์ ลีก รอบก่อนรองชนะเลิศ นัด 2 สัปดาห์นี้\n\nซาเน่ จะเหลือสัญญาอีกเพียงปีเดียวเท่านั้น และตกเป็นข่าวว่าได้รับความสนใจจากลิเวอร์พูล ที่มองหาตัวแทน โมฮาเหม็ด ซาลาห์\n\n""เสือใต้"" เตรียมยื่นข้อเสนอสัญญาใหม่ให้แก่เขา แต่ตอนนี้ ทั้งสองฝ่ายยังไม่ได้บรรลุข้อตกลงอะไรกัน\n\nรายงานอ้างว่า บาเยิร์น วางกำหนดพบตัวแทนของดาวเตะทีมชาติเยอรมนี ในอีกไม่กี่วันข้างหน้า หลังจบเกมที่อัลลิอันซ์ อารีน่า วันพุธนี้\n\nแม้มีการเจรจาอย่างไม่เป็นทางการก่อนหน้า ..

In [24]:
%%time
def get_post_head_desc(url):
    soup = get_soup(url)
    return {'post_head': soup.find('div', class_='post_head').text.strip(),
            'post_desc': soup.find('div', class_='post_desc').text.strip()}

pd.DataFrame(df.url.map(get_post_head_desc).tolist())

CPU times: user 2.04 s, sys: 27.4 ms, total: 2.07 s
Wall time: 12 s


,post_head,post_desc
0,ชาบี้มั่นใจ บาร์ซ่าบู๊ PSG เร้าใจไม่แพ้นัดแรก,"ชาบี้เอร์นานเดซ บอสใหญ่บาร์เซโลน่า เชื่อว่า เกมปะทะปารีส แซงต์ แชร์กแมง ในยูฟ่า แชมเปี้ยนส์ลีก รอบก่อนรองชนะเลิศ นัดสอง จะต้องเร้าใจไม่แพ้เกมแรกที่พวกเขาบุกไปปราบทีมแชมป์ลีกเอิงถึงถิ่น 3-2\n\nทีมของ ชาบี้ สร้างผลงานยอดเยี่ยมบุกไปยัดเยียดความปราชัยให้ เปแอสเช ได้ถึงฝรั่งเศส 3-2 เมื่อสัปดาห์ที่แล้ว สถานการณ์ได้เปรียบที่จะตบเท้าเข้ารอบรองชนะเลิศได้เป็นครั้งแรกนับตั้งแต่ปี 2019\n\n""เราไม่ใช่ทีมที่นำแล้วอุด เราต้องการแย่งบอลจากเปแอสเชและเป็นผู้ชนะในเกมนี้"" ชาบี้ กล่าวในเพรส คอนเฟอเรนซ์ ก่อนเกมสำคัญวันอังคารนี้\n\n""ผมหวังว่าเราจะอยู่ในจุดที่ดีที่สุดของตัวเอง พวกเขาจะทดสอบขีดจำกัดของเรา ผมคิดว่า..."
1,"สิงห์ใช้ออพชั่นขยายสัญญา 'เอนโซ, มูดรีค' อยู่ยาวยัน 30","รายงานข่าวจาก The Standard เปิดเผยว่า ""สิงห์บลูส์"" เชลซี เปิดใช้ออพชั่นขยายสัญญาระยะยาวให้กับ เอนโซ แฟร์นานเดซ และ มิคายโล มูดรีค ยาวไปจนถึงอายุ 30 ปี\n\n\nเชลซี เซ็นสัญญากับ เอนโซ ด้วยค่าตัวที่แพงเป็นสถิติอังกฤษที่ 107 ล้านปอนด์ จากเบนฟิก้า เมื่อมกราคม ปี 2023 ขณะที่ มูดรีค ย้ายมาจากชัคตาร์ โดเนสก์ ค่าตัว 88 ล้านปอนด์ในตลาดรอบเดียวกัน\n\nรายงานจาก The Standard อ้างอิงเอกสารของเอฟเอ ที่เปิดเผยว่า เอนโซ เหลือสัญญาอีก 8 ปีกับค่าเหนื่อย 180,000 ปอนด์ต่อสัปดาห์ ซึ่งสัญญาของเขาจะหมดลงในปี 2032 หรือตอนอายุ 31 ปี\n\nขณะที่ มูดรีค เหลือสัญญา 7 ปี กับค่าเหนื่อย 87,000 ปอนด์ต่อสัปดาห์ และสตาร์ยูเครน..."
2,เผย ETH เรียก 'การ์นาโช่' คุยส่วนตัวเคลียร์ดราม่ากดไลค์,รายงานข่าวจาก Manchester Evening News เปิดเผยว่า เอริค เทน ฮาก ได้พูดคุยส่วนตัวกับ อเลฮานโดร การ์นาโช่ หลังดาวรุ่งวัย19 ปี ไปกดไลค์ถูกใจทวิตวิจารณ์บอสชาวดัตช์หลังเกมเสมอบอร์นมัธ 2-2 เมื่อคืนวันเสาร์ที่ผ่านมา\n\n\nช่วงพักครึ่งของเกมดังกล่าว เทน ฮาก ตัดสินใจเปลี่ยนตัวเอา อาหมัด ดิยาลโล่ ลงไปแทน การ์นาโช่ และทีมก็ดูดีขึ้นขึ้นเล็กน้อยในครึ่งหลัง\n\nเมื่อถามว่าทำไมเขาถึงถอด การ์นาโช่ ออกตอนพักครึ่ง เทน ฮาก ตอบในทำนองโยนความผิดให้ปีกดาวรุ่งผู้นี้\n\nจากนั้นหลังจบเกม โกลด์บริดจ์ ที่มักจะวิจารณ์ทีมตัวเองอยู่เสมอไม่พลาดที่จะจัดหนักใส่ เทน ฮาก โดยเฉพาะการถอด การ์นาโช่ ออก และปีกวัย 19 ปีไปกดไลค์ทั้ง...
3,แชมป์เปี้ยนชิพเลือก 'แม็คเคนน่า' กุนซือแห่งปี,"เคียแรน แม็คเคนน่า อดีตผู้ช่วย แมนเชสเตอร์ ยูไนเต็ด ได้รับเลือกเป็นกุนซือยอดเยี่ยมประจำซีซั่น แชมป์เปี้ยนชิพ จากผลงานคุม อิปสวิช ทาวน์ รั้งจ่าฝูง ขณะที่รางวัลฝั่งนักเตะ ตกเป็นของ คริสเซนซิโอ้ ซัมเมอร์วิลล์ แนวรุก ลีดส์ ยูไนเต็ด\n\nแม็คเคนน่า นั่งแท่นผู้จัดการ ""ม้าขาว"" แบบเต็มตัวเป็นฤดูกาลที่สอง และกำลังทำผลงานนำจ่าฝูงเหนือ เลสเตอร์ ซิตี้ 1 แต้ม ขณะที่การแข่งเหลือ 3 นัดสุดท้าย ลุ้นเลื่อนชั้นสองซีซั่นติด ภายหลังเพิ่งขึ้นมาจาก ลีก วัน \n\nผลงานดังกล่าวทำให้เฮดโค้ชชาวไอร์แลนด์เหนือ ได้รับรางวัลประจำปีเหนือผู้ท้าชิงอย่าง ดาเนียล ฟาร์เก้ ของ ลีดส์ ยูไนเต็ด และ เลียม โรเซเนียร์ จาก ฮัลล์ ซิตี้\n\..."
4,ฝันเป็นจริง! แฟนคลับนิวชาวไทยถูกเชิญดูบอลถึง เซนต์ เจมส์ พาร์ค สุดฟิน,กลุ่มแฟนคลับทางการของ นิวคาสเซิล ยูไนเต็ด ประเทศไทย (NUFCTH) บินไปดูพรีเมียร์ลีกของทีมรักแบบติดขอบสนาม เมื่อสุดสัปดาห์ที่ผ่านมา\n\nFUN ชวนแฟน ๆ นิวคาสเซิลจากเมืองไทย มาร่วมชมเกมระหว่าง “สาลิกาดง” กับ ท็อตแนม ฮ็อตสเปอร์ ที่สนาม “เซนต์ เจมส์ พาร์ค” ซึ่งผลการแข่งขันจบลงที่เจ้าบ้านเปิดรังถล่มเอาชนะทีมเยือนไปเละเทะ 4-0\n\nคุณวิน หนึ่งในแฟนคลับที่ได้ไปสัมผัสประสบการณ์ฟุตบอลผู้ดีแบบสด ๆ เปิดใจหลังจบเกมว่า “ดีใจมาก ๆ ครับที่ได้มาดู นิวคาสเซิล ถึงขอบสนามเป็นครั้งแรก เป็นประสบการณ์ที่สุดยอดมาก ๆ ผมยังขนลุกอยู่เลยกับบรรยากาศใน เซนต์ เจมส์ พาร์ค ต้องขอขอบคุณสปอนเซอร์ใจดีอย่าง FUN ที่มอบโอกาสนี้ให้กับพ...
5,เตรียมตัวมา 2 ปี! ‘โค้ชหระ’ มอง U23 ทุกทีมมีโอกาสออกได้ทั้งสามหน้า,"อิสสระ ศรีทะโร แถลงความพร้อม ทีมชาติไทย U23 ย้ำเป้าหมายมาเพื่อคว้าตั๋วไปโอลิมปิก แต่โฟกัสเกมต่อเกม มองทุกทีมมีโอกาส ออกได้ทั้งสามหน้า\n\nวันที่ 15 เมษายน 2567 เวลา 17.30 น. ตามเวลาประเทศไทย ณ ห้องแถลงข่าว คอลิฟา อินเตอร์เนชันแนล สเตเดียม สมาพันธ์ฟุตบอลแห่งเอเชีย หรือ เอเอฟซี จัดงานแถลงข่าวก่อนการแข่งขันฟุตบอล AFC U23 Asian Cup QATAR 2024 กลุ่มซี \n\nการแถลงข่าวครั้งนี้ประกอบไปด้วย 4 ชาติในกลุ่ม ซี ประกอบไปด้วย ซาอุดีอาระเบีย, อิรัก, ไทย และ ทาจิกิสถาน โดย ไทยได้ส่ง อิสสระ ศรีทะโร หัวหน้าผู้ฝึกสอน เป็นตัวแทน\n\nโดย อิสสระ ศรีทะโร หัวหน้าผู้ฝึกสอนทีมชาติไทย U23 กล่าวว่า ""ทีมชาติไทยมีการเตรีย..."
6,เช็คความ

In [25]:
%%time
res = []
for url in df.url.tolist():
     res.append(get_post_head_desc(url))
pd.DataFrame(res)

CPU times: user 1.28 s, sys: 26.1 ms, total: 1.3 s
Wall time: 10.5 s


,post_head,post_desc
0,ชาบี้มั่นใจ บาร์ซ่าบู๊ PSG เร้าใจไม่แพ้นัดแรก,"ชาบี้เอร์นานเดซ บอสใหญ่บาร์เซโลน่า เชื่อว่า เกมปะทะปารีส แซงต์ แชร์กแมง ในยูฟ่า แชมเปี้ยนส์ลีก รอบก่อนรองชนะเลิศ นัดสอง จะต้องเร้าใจไม่แพ้เกมแรกที่พวกเขาบุกไปปราบทีมแชมป์ลีกเอิงถึงถิ่น 3-2\n\nทีมของ ชาบี้ สร้างผลงานยอดเยี่ยมบุกไปยัดเยียดความปราชัยให้ เปแอสเช ได้ถึงฝรั่งเศส 3-2 เมื่อสัปดาห์ที่แล้ว สถานการณ์ได้เปรียบที่จะตบเท้าเข้ารอบรองชนะเลิศได้เป็นครั้งแรกนับตั้งแต่ปี 2019\n\n""เราไม่ใช่ทีมที่นำแล้วอุด เราต้องการแย่งบอลจากเปแอสเชและเป็นผู้ชนะในเกมนี้"" ชาบี้ กล่าวในเพรส คอนเฟอเรนซ์ ก่อนเกมสำคัญวันอังคารนี้\n\n""ผมหวังว่าเราจะอยู่ในจุดที่ดีที่สุดของตัวเอง พวกเขาจะทดสอบขีดจำกัดของเรา ผมคิดว่า..."
1,"สิงห์ใช้ออพชั่นขยายสัญญา 'เอนโซ, มูดรีค' อยู่ยาวยัน 30","รายงานข่าวจาก The Standard เปิดเผยว่า ""สิงห์บลูส์"" เชลซี เปิดใช้ออพชั่นขยายสัญญาระยะยาวให้กับ เอนโซ แฟร์นานเดซ และ มิคายโล มูดรีค ยาวไปจนถึงอายุ 30 ปี\n\n\nเชลซี เซ็นสัญญากับ เอนโซ ด้วยค่าตัวที่แพงเป็นสถิติอังกฤษที่ 107 ล้านปอนด์ จากเบนฟิก้า เมื่อมกราคม ปี 2023 ขณะที่ มูดรีค ย้ายมาจากชัคตาร์ โดเนสก์ ค่าตัว 88 ล้านปอนด์ในตลาดรอบเดียวกัน\n\nรายงานจาก The Standard อ้างอิงเอกสารของเอฟเอ ที่เปิดเผยว่า เอนโซ เหลือสัญญาอีก 8 ปีกับค่าเหนื่อย 180,000 ปอนด์ต่อสัปดาห์ ซึ่งสัญญาของเขาจะหมดลงในปี 2032 หรือตอนอายุ 31 ปี\n\nขณะที่ มูดรีค เหลือสัญญา 7 ปี กับค่าเหนื่อย 87,000 ปอนด์ต่อสัปดาห์ และสตาร์ยูเครน..."
2,เผย ETH เรียก 'การ์นาโช่' คุยส่วนตัวเคลียร์ดราม่ากดไลค์,รายงานข่าวจาก Manchester Evening News เปิดเผยว่า เอริค เทน ฮาก ได้พูดคุยส่วนตัวกับ อเลฮานโดร การ์นาโช่ หลังดาวรุ่งวัย19 ปี ไปกดไลค์ถูกใจทวิตวิจารณ์บอสชาวดัตช์หลังเกมเสมอบอร์นมัธ 2-2 เมื่อคืนวันเสาร์ที่ผ่านมา\n\n\nช่วงพักครึ่งของเกมดังกล่าว เทน ฮาก ตัดสินใจเปลี่ยนตัวเอา อาหมัด ดิยาลโล่ ลงไปแทน การ์นาโช่ และทีมก็ดูดีขึ้นขึ้นเล็กน้อยในครึ่งหลัง\n\nเมื่อถามว่าทำไมเขาถึงถอด การ์นาโช่ ออกตอนพักครึ่ง เทน ฮาก ตอบในทำนองโยนความผิดให้ปีกดาวรุ่งผู้นี้\n\nจากนั้นหลังจบเกม โกลด์บริดจ์ ที่มักจะวิจารณ์ทีมตัวเองอยู่เสมอไม่พลาดที่จะจัดหนักใส่ เทน ฮาก โดยเฉพาะการถอด การ์นาโช่ ออก และปีกวัย 19 ปีไปกดไลค์ทั้ง...
3,แชมป์เปี้ยนชิพเลือก 'แม็คเคนน่า' กุนซือแห่งปี,"เคียแรน แม็คเคนน่า อดีตผู้ช่วย แมนเชสเตอร์ ยูไนเต็ด ได้รับเลือกเป็นกุนซือยอดเยี่ยมประจำซีซั่น แชมป์เปี้ยนชิพ จากผลงานคุม อิปสวิช ทาวน์ รั้งจ่าฝูง ขณะที่รางวัลฝั่งนักเตะ ตกเป็นของ คริสเซนซิโอ้ ซัมเมอร์วิลล์ แนวรุก ลีดส์ ยูไนเต็ด\n\nแม็คเคนน่า นั่งแท่นผู้จัดการ ""ม้าขาว"" แบบเต็มตัวเป็นฤดูกาลที่สอง และกำลังทำผลงานนำจ่าฝูงเหนือ เลสเตอร์ ซิตี้ 1 แต้ม ขณะที่การแข่งเหลือ 3 นัดสุดท้าย ลุ้นเลื่อนชั้นสองซีซั่นติด ภายหลังเพิ่งขึ้นมาจาก ลีก วัน \n\nผลงานดังกล่าวทำให้เฮดโค้ชชาวไอร์แลนด์เหนือ ได้รับรางวัลประจำปีเหนือผู้ท้าชิงอย่าง ดาเนียล ฟาร์เก้ ของ ลีดส์ ยูไนเต็ด และ เลียม โรเซเนียร์ จาก ฮัลล์ ซิตี้\n\..."
4,ฝันเป็นจริง! แฟนคลับนิวชาวไทยถูกเชิญดูบอลถึง เซนต์ เจมส์ พาร์ค สุดฟิน,กลุ่มแฟนคลับทางการของ นิวคาสเซิล ยูไนเต็ด ประเทศไทย (NUFCTH) บินไปดูพรีเมียร์ลีกของทีมรักแบบติดขอบสนาม เมื่อสุดสัปดาห์ที่ผ่านมา\n\nFUN ชวนแฟน ๆ นิวคาสเซิลจากเมืองไทย มาร่วมชมเกมระหว่าง “สาลิกาดง” กับ ท็อตแนม ฮ็อตสเปอร์ ที่สนาม “เซนต์ เจมส์ พาร์ค” ซึ่งผลการแข่งขันจบลงที่เจ้าบ้านเปิดรังถล่มเอาชนะทีมเยือนไปเละเทะ 4-0\n\nคุณวิน หนึ่งในแฟนคลับที่ได้ไปสัมผัสประสบการณ์ฟุตบอลผู้ดีแบบสด ๆ เปิดใจหลังจบเกมว่า “ดีใจมาก ๆ ครับที่ได้มาดู นิวคาสเซิล ถึงขอบสนามเป็นครั้งแรก เป็นประสบการณ์ที่สุดยอดมาก ๆ ผมยังขนลุกอยู่เลยกับบรรยากาศใน เซนต์ เจมส์ พาร์ค ต้องขอขอบคุณสปอนเซอร์ใจดีอย่าง FUN ที่มอบโอกาสนี้ให้กับพ...
5,เตรียมตัวมา 2 ปี! ‘โค้ชหระ’ มอง U23 ทุกทีมมีโอกาสออกได้ทั้งสามหน้า,"อิสสระ ศรีทะโร แถลงความพร้อม ทีมชาติไทย U23 ย้ำเป้าหมายมาเพื่อคว้าตั๋วไปโอลิมปิก แต่โฟกัสเกมต่อเกม มองทุกทีมมีโอกาส ออกได้ทั้งสามหน้า\n\nวันที่ 15 เมษายน 2567 เวลา 17.30 น. ตามเวลาประเทศไทย ณ ห้องแถลงข่าว คอลิฟา อินเตอร์เนชันแนล สเตเดียม สมาพันธ์ฟุตบอลแห่งเอเชีย หรือ เอเอฟซี จัดงานแถลงข่าวก่อนการแข่งขันฟุตบอล AFC U23 Asian Cup QATAR 2024 กลุ่มซี \n\nการแถลงข่าวครั้งนี้ประกอบไปด้วย 4 ชาติในกลุ่ม ซี ประกอบไปด้วย ซาอุดีอาระเบีย, อิรัก, ไทย และ ทาจิกิสถาน โดย ไทยได้ส่ง อิสสระ ศรีทะโร หัวหน้าผู้ฝึกสอน เป็นตัวแทน\n\nโดย อิสสระ ศรีทะโร หัวหน้าผู้ฝึกสอนทีมชาติไทย U23 กล่าวว่า ""ทีมชาติไทยมีการเตรีย..."
6,เช็คความ

### Dynamic Web

ในบางกรณี เว็บไซต์จำเป็นต้องให้เราทำกิจกรรมอะไรบางอย่างเพื่อเข้าถึงหน้าที่เราต้องการ scrape เช่น ต้องล็อกอินก่อน, ต้องกดบางปุ่มก่อน เป็นต้น เราเรียกเว็บไซต์ที่มีการเปลี่ยนแปลงจากการทำกิจกรรมอะไรบางอย่างเหล่านี้ว่า Dynamic Web

ในกรณี soccersuck.com สมมุติว่าเราไม่รู้ url ของหน้าต่อไป จำเป็นต้องเขียนโปรแกรมให้กดหน้าต่อไปให้ เราจะใช้ [selenium](https://selenium-python.readthedocs.io) มาช่วยทำกิจกรรมเหล่านี้ (กดหน้าถัดไป)

วิธีลง selenium บน colab จาก [Chalach Monkhontirapat](https://medium.com/equinox-blog/%E0%B8%81%E0%B8%B2%E0%B8%A3%E0%B9%83%E0%B8%8A%E0%B9%89-selenium-%E0%B8%9A%E0%B8%99-google-colaboratory-984739ed44e5)

In [26]:
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF


Executing: /tmp/apt-key-gpghome.dnYJvxHmk2/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
Executing: /tmp/apt-key-gpghome.eve4Xb5yXy/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
Executing: /tmp/apt-key-gpghome.VeHUFoCW8I/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
gpg: cannot open '/dev/tty': No such device or address
gpg: [stdout]: write error: Broken pipe
gpg: filter_flush failed on c

In [27]:
!apt-get update
!apt-get install chromium chromium-driver
!pip3 -q install selenium
exit() #reset runtime เพื่อให้ package ใช้งานได้อย่างถูกต้อง

Hit:1 http://deb.debian.org/debian buster InRelease
Hit:2 http://deb.debian.org/debian buster-updates InRelease
Hit:3 http://deb.debian.org/debian-security buster/updates InRelease
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:10 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:13 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency t

In [28]:
import pandas as pd

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

#### Selenium ก็ Scrape Static Web ได้

In [29]:
#ติดตั้ง web driver เพื่อจำลองผู้ใช้เว็บไซต์
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

#ฟังชั่นเปลี่ยน soup เป็น dataframe
def get_df(soup):
    return pd.DataFrame([{'headline_abridged': i.text.strip(),
                          'url': i.find('a').get('href')} for i in soup.find_all('div', class_='oldnew_p_tr')])

driver = webdriver.Chrome(options=chrome_options)

In [30]:
#เข้าไปหน้าข่าวเพื่อเก็บ url เหมือนเดิม
url = f'https://www.soccersuck.com/boards/oldnews'
driver.get(url)
soup = BeautifulSoup(driver.page_source)
driver.close()

df = pd.DataFrame([
{'headline_abridged': i.text.strip(),
 'url': i.find('a').get('href')} for i in soup.find_all('div', class_='oldnew_p_tr')]).head(10)
df

,headline_abridged,url
0,ชาบี้มั่นใจ บาร์ซ่าบู๊ PSG เร้าใจไม่แพ้นัดแรก,https://www.soccersuck.com/boards/topic/2431838
1,"สิงห์ใช้ออพชั่นขยายสัญญา 'เอนโซ, มูดรีค' อยู่ยาวยั...",https://www.soccersuck.com/boards/topic/2431832
2,เผย ETH เรียก 'การ์นาโช่' คุยส่วนตัวเคลียร์ดราม่าก...,https://www.soccersuck.com/boards/topic/2431825
3,แชมป์เปี้ยนชิพเลือก 'แม็คเคนน่า' กุนซือแห่งปี,https://www.soccersuck.com/boards/topic/2431800
4,ฝันเป็นจริง! แฟนคลับนิวชาวไทยถูกเชิญดูบอลถึง เซนต์ เจมส์ พาร...,https://www.soccersuck.com/boards/topic/2431798
5,เตรียมตัวมา 2 ปี! ‘โค้ชหระ’ มอง U23 ทุกทีมมีโอกาสออกได้ทั้งส...,https://www.soccersuck.com/boards/topic/2431793
6,เช็คความพร้อม! ‘อิสสระ-อิชิอิ’ ตรวจสนาม ไทย บู๊ อิรัก ประเดิ...,https://www.soccersuck.com/boards/topic/2431751
7,กิจวัตรใหม่! ขุนแจ็ค' ควงน้องซาช่าซาวน่าทุกคืน,https://www.soccersuck.com/boards/topic/2431747
8,หงส์จ้องอยู่! พี่เสือนัดตัวแทน 'ซาเน่' ถกสัญญาใหม่...,https://www.soccersuck.com/boards/topic/2431744
9,เจ็บเล็กน้อย...ผีปัด 'เดอะ หมุน' เกาเหลา ETH,https://www.soccersuck.com/boards/topic/2431738


#### Selenium สำหรับ Dynamic Web

In [31]:
#ติดตั้ง web driver เพื่อจำลองผู้ใช้เว็บไซต์
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

#ฟังชั่นเปลี่ยน soup เป็น dataframe
def get_df(soup):
    return pd.DataFrame([{'headline_abridged': i.text.strip(),
                          'url': i.find('a').get('href')} for i in soup.find_all('div', class_='oldnew_p_tr')])

driver = webdriver.Chrome(options=chrome_options)

In [32]:
dfs = []

#เปิดหน้าเว็บตาม url
url = f'https://www.soccersuck.com/boards/oldnews'
driver.get(url)
print(f'Opened {url}')

#เก็บข้อมูล soup และ dataframe จากหน้า 1
soup = BeautifulSoup(driver.page_source)
d = get_df(soup)
d['page'] = '1'
dfs.append(d)
print(f'Saved soup and dataframe from page 1')

#เราจะกดปุ่มหน้า 2 และ 3
for i in ['2','3']:

    #เช็คดูว่าปุ่มหน้าที่จะกด ('2' และ '3') โหลดแล้วหรือยัง
    try:
        element = WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.LINK_TEXT, i))
        )
        print(f'Pagination button {i} found!')
    except:
        print(f'Pagination button {i} NOT found')

    #คลิกไปหน้าถัดไป
    link = driver.find_element(By.LINK_TEXT, i)
    link.click()
    print(f'Clicked pagination button {i}')

    #เก็บข้อมูล soup และ dataframe จากหน้า i
    soup = BeautifulSoup(driver.page_source)
    d = get_df(soup)
    d['page'] = i
    dfs.append(d)
    print(f'Saved soup and dataframe from page {i}')

#ปิดหน้า web
driver.close()

Opened https://www.soccersuck.com/boards/oldnews
Saved soup and dataframe from page 1
Pagination button 2 found!
Clicked pagination button 2
Saved soup and dataframe from page 2
Pagination button 3 found!
Clicked pagination button 3
Saved soup and dataframe from page 3


In [33]:
df = pd.concat(dfs).reset_index(drop=True)
df.tail()

,headline_abridged,url,page
298,จอมยัวะระวัง! ยูฟ่าออกโรงเตือนชาติแข่งยูโรห้ามโวยเปา,https://www.soccersuck.com/boards/topic/2429701,3
299,'เมอร์ฟี่' เปิดอกติดโคเคนหนักหลังแขวนสตั๊ด,https://www.soccersuck.com/boards/topic/2429700,3
300,ซีอีโอพรีเมียร์หวั่นกฎหมายรัฐส่งผลกระทบลีก,https://www.soccersuck.com/boards/topic/2429692,3
301,"บาร์ซ่าหนีบ 'เฟรนกี้,กาบี้' ดวล PSG, หมีไร้ '...",https://www.soccersuck.com/boards/topic/2429682,3
302,'โชโล่' รับเจอเสือเหลืองไม่ใช่งานง่าย,https://www.soccersuck.com/boards/topic/2429679,3


## สร้างขึ้นมาเอง

### Annotation, Annotation, Annotation

วิธีที่ตรงไปตรงมาที่สุดคือการเก็บและ label ข้อมูลด้วยมนุษย์ คุณสามารถชวนเพื่อนๆมาทำ หรือแม้แต่จ้างบริษัทมืออาชีพ แน่นอนว่าข้อมูลยิ่งมากยิ่งดี แต่หลายครั้งแค่ 1,000 ตัวอย่างก็พอให้คุณเริ่มทำ ML model ได้แล้ว

ยกตัวอย่างเช่น [@wannaphong](https://github.com/wannaphong) ผู้ก่อตั้ง [PyThaiNLP](https://github.com/PyThaiNLP/pythainlp) ผู้เชี่ยวชาญด้านการสร้างชุดข้อมูลด้วยมือ [@wannaphong](https://github.com/wannaphong) สร้าง

* [thai-named-entity-recognition-data](https://github.com/PyThaiNLP/thai-named-entity-recognition-data/) - ชุดข้อมูล named entity recognition ที่ต่อมารวมกับชุดข้อมูลอีกชุด (ที่ [@wannaphong](https://github.com/wannaphong) ทำการตรวจสอบคุณภาพด้วยมืออีกรอบ) กลายเป็น [ThaiNER](https://github.com/wannaphong/thai-ner) ที่เป็นหนึ่งใน benchmark ที่ใหญ่ที่สุดสำหรับ NER ภาษาไทย ทดลองใช้ได้ที่ [HuggingFace Hub](https://huggingface.co/pythainlp/thainer-corpus-v2-base-model)

* [spelling-check](https://github.com/PyThaiNLP/spelling-check) - ชุดข้อมูลตรวจสอบการสะกดคำภาษาไทยกว่า 1,800 ประโยค; ใช้สำหรับฟังชั่นแก้คำผิดของ [PyThaiNLP](https://github.com/PyThaiNLP/pythainlp)

* [Thai-Lao-Parallel-Corpus](https://github.com/PyThaiNLP/Thai-Lao-Parallel-Corpus) - แปลภาษาไทย-ลาวประมาณ 260 ประโยค

* [thai-sentence](https://github.com/PyThaiNLP/Thai-sentence) - ประโยคภาษาไทยเขียนเองประมาณ 1,000 ประโยค

* [thai-synonym](https://github.com/PyThaiNLP/thai-synonym/) - คำพ้องความหมายประมาณ 100 คำ

ข้อดีของการทำแบบนี้คือคุณจะเข้าใจชุดข้อมูลของคุณได้ดียิ่งขึ้น และยิ่งถ้าคุณเปิดเผยชุดข้อมูลเป็นสาธารณะก็อาจจะมีคนมาช่วยเพิ่มข้อมูลให้คุณอีกด้วย

### สร้าง Tabular Data เพิ่มด้วย ML Models

เริ่มจากถ้าเรามีข้อมูล tabular อยู่บ้าง แต่ต้องการเพิ่มเพื่อให้โมเดลมีประสิทธิภาพขึ้น เราสามารถสร้างชุดข้อมูลเพิ่มด้วยเทคนิค เช่น [CTGAN](https://arxiv.org/abs/1907.00503) เรียกใช้จาก [sdv](https://sdv.dev/SDV/user_guides/single_table/ctgan.html)

เราทดลองใช้ [CTGAN](https://arxiv.org/abs/1907.00503) สร้างชุดข้อมูลเพิ่มสำหรับโมเดล [xgboost](https://xgboost.readthedocs.io/en/stable/) ในบทที่ 1

จำแนกการตอบสนองของลูกค้ากับแคมเปญที่บริษัทจัดด้วยชุดข้อมูล [Retail Transaction Data](https://www.kaggle.com/regivm/retailtransactiondata) จัดทำโดย []() ดูตัวอย่างการวิเคราะห์เต็มๆได้ที่ [Campaign Response Model - Example.ipynb](https://colab.research.google.com/drive/1eTVlSYcwUYkYDW6sPTY8XVO1QMhr-DNT?usp=sharing)

In [34]:
#download data
!wget https://github.com/ai-builders/curriculum/raw/main/data/Retail_Data_Transactions.csv
!wget https://github.com/ai-builders/curriculum/raw/main/data/Retail_Data_Response.csv

--2024-04-17 13:46:52--  https://github.com/ai-builders/curriculum/raw/main/data/Retail_Data_Transactions.csv
Resolving github.com (github.com)... 140.82.116.4
Connecting to github.com (github.com)|140.82.116.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/ai-builders/curriculum/main/data/Retail_Data_Transactions.csv [following]
--2024-04-17 13:46:52--  https://raw.githubusercontent.com/ai-builders/curriculum/main/data/Retail_Data_Transactions.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2633502 (2.5M) [text/plain]
Saving to: ‘Retail_Data_Transactions.csv’

Retail_Data_Transac 100%[===================>]   2.51M  --.-KB/s    in 0.04s   

2024-04-17 13:46:53 (59.0 MB/s) - ‘Retail_Data_Tr

In [35]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score

#จัดการข้อมูลให้อยู่ในรูปแบบ (X, y)
df_response = pd.read_csv('Retail_Data_Response.csv')
df_trans = pd.read_csv('Retail_Data_Transactions.csv', parse_dates=['trans_date'])

campaign_date = datetime(2015,3,17)
df_trans['age'] = (campaign_date - df_trans.trans_date).dt.days

df = df_trans.groupby('customer_id').agg(
    recency=('age', 'min'),
    frequency=('customer_id', 'count'),
    monetary=('tran_amount', 'sum'),
    tenure=('age', 'max'),
    length_of_stay=('age', lambda x: x.max() - x.min())
)
df['ticket_size'] = df.monetary / df.frequency
df['std_ticket_size'] = df_trans.groupby('customer_id').tran_amount.std()
df['cv_ticket_size'] = df.std_ticket_size / df.ticket_size

# the number of months visited
active_months = df_trans.groupby(['customer_id', pd.Grouper(key='trans_date',freq='M')]).size().reset_index()
active_months = active_months.groupby('customer_id').trans_date.nunique()
df['active_months'] = active_months
df['avg_spend_m'] = df.monetary / df.active_months
df['avg_visit_m'] = df.frequency / df.active_months
# done with active_months, then drop it
df = df.drop(columns='active_months')
df['avg_tte'] = df.length_of_stay / (df.frequency - 1)

#all data
all_data = df_response.join(df, how='inner', on='customer_id').set_index('customer_id')

#split ข้อมูลเป็น train, validation และ test
random_state = 1024
test_size = 0.2
val_size = 0.2

X = all_data.drop(columns='response')
y = all_data.response

X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=test_size,
                                                            stratify=y, random_state=random_state)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=val_size/(1-test_size),
                                                  stratify=y_train_val, random_state=random_state)

train_data = X_train.join(y_train)
val_data = X_val.join(y_val)
test_data = X_test.join(y_test)

train_data.shape, val_data.shape, test_data.shape

((4130, 12), (1377, 12), (1377, 12))

เราสร้างชุดข้อมูลเพิ่มจาก `train data` ด้วย CTGAN (ระวังอย่าสร้างจาก validation หรือ test sets ไม่งั้นจะเปิด data leakage ("โกงข้อสอบ")

In [36]:
from sdv.tabular import CTGAN

#สร้างโมเดล CTGAN เพื่อเรียนรู้จากข้อมูล
model = CTGAN()
model.fit(train_data)

#สร้างชุดข้อมูลใหม่มา 4000 ตัวอย่าง (เดิม training set มี 4,130 ตัวอย่าง)
new_train_data = model.sample(4000)
new_train_data.tail()

/usr/local/lib/python3.10/dist-packages/rdt/transformers/base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missing_values' parameter. Please switch to using the 'missing_value_generation' parameter to select your strategy.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/rdt/transformers/numerical.py:112: UserWarning: No rounding scheme detected for column 'ticket_size'. Data will not be rounded.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/rdt/transformers/numerical.py:112: UserWarning: No rounding scheme detected for column 'std_ticket_size'. Data will not be rounded.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/rdt/transformers/numerical.py:112: UserWarning: No rounding scheme detected for column 'cv_ticket_size'. Data will not be rounded.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/rdt/transformers/numerical.py:112: UserWarning: No rounding scheme detected for column 'avg_spend_m'. Data will not be rounded.
 

,recency,frequency,monetary,tenure,length_of_stay,ticket_size,std_ticket_size,cv_ticket_size,avg_spend_m,avg_visit_m,avg_tte,response
3995,22,30,1149,1385,1191,64.518566,21.257720,0.247731,100.119964,1.554338,51.293731,0
3996,263,17,464,1261,1080,40.304676,16.652414,0.551617,53.321190,1.067304,118.852161,0
3997,43,8,342,929,1107,38.568985,19.500980,0.616033,51.002508,1.013396,150.232176,0
3998,60,20,1342,1272,1190,65.125036,22.935158,0.319800,85.621056,1.243165,90.346892,0
3999,8,28,1558,1374,1109,62.389980,15.965497,0.288013,90.716855,1.217704,65.952256,0


In [37]:
#ประเมินว่าสร้างข้อมูลปลอมได้ดีแค่ไหน; ค่ายิ่งสูงยิ่งดี
from sdv.evaluation import evaluate

evaluate(new_train_data, real_data=train_data)

0.8895554378531075

In [38]:
#สร้าง X_train_new และ y_train_new ด้วยการเพิ่มข้อมูลปลอมเข้าไป
X_train_new = pd.concat([X_train,new_train_data.drop(columns='response')])
y_train_new = pd.concat([y_train,new_train_data.response])

In [39]:
#สร้างโมเดลต้นไม้ด้วย xgboost (gradient boosted trees)
import xgboost as xgb

#เทรนด้วยข้อมูลจริง
xgb_params = {'objective': 'binary:logistic',
              'random_state': 1024,
              'eval_metric': 'auc',
              'early_stopping_rounds': 10,
              'scale_pos_weight' : (len(y_train) - sum(y_train)) / sum(y_train),
              'max_depth': 3,
              'booster':'gbtree',
              }

model = xgb.XGBClassifier(**xgb_params)
model = model.fit(X_train,
                  y_train,
                  eval_set=[(X_val,y_val)],
                  verbose=False)

#เทรนด้วยข้อมูลจริง + ข้อมูลปลอม
xgb_params_new = {'objective': 'binary:logistic',
              'random_state': 1024,
              'eval_metric': 'auc',
              'early_stopping_rounds': 10,
              'scale_pos_weight' : (len(y_train_new) - sum(y_train_new)) / sum(y_train_new),
              'max_depth': 3,
              'booster':'gbtree',
              }

model_new = xgb.XGBClassifier(**xgb_params_new)
model_new = model_new.fit(X_train_new,
                  y_train_new,
                  eval_set=[(X_val,y_val)],
                  verbose=False)

จะเห็นได้ว่าเมื่อเพิ่มข้อมูลปลอมเข้าไปแล้ว micro-averaged F1 และ accuracy ดีขึ้น แต่ AUC แย่ลง เพราะฉะนั้นการที่เราจะเพิ่มข้อมูลปลอมเข้าไปเทรนจำเป็นต้องคำนึงถึงว่าเราต้องการวัดผลของโมเดลด้วย metric ใด

In [40]:
#รายงานผลโมเดลเทรนบนข้อมูลจริง
print(f'AUC: {roc_auc_score(y_test, model.predict_proba(X_test)[:,1])}')
print(classification_report(y_test,model.predict_proba(X_test)[:,1].round()))

AUC: 0.7742061717352415
              precision    recall  f1-score   support

           0       0.97      0.63      0.77      1248
           1       0.18      0.78      0.29       129

    accuracy                           0.65      1377
   macro avg       0.57      0.71      0.53      1377
weighted avg       0.89      0.65      0.72      1377



In [41]:
#รายงานผลโมเดลเทรนบนข้อมูลจริง+ข้อมูลปลอม
print(f'AUC: {roc_auc_score(y_test, model_new.predict_proba(X_test)[:,1])}')
print(classification_report(y_test,model_new.predict_proba(X_test)[:,1].round()))

AUC: 0.7611993142516398
              precision    recall  f1-score   support

           0       0.97      0.59      0.73      1248
           1       0.17      0.83      0.29       129

    accuracy                           0.61      1377
   macro avg       0.57      0.71      0.51      1377
weighted avg       0.90      0.61      0.69      1377



### สร้าง Texts จาก ML Models

สำหรับโมเดล NLP นั้นโชคดีที่ในอินเตอร์เน็ตมีข้อความมากมายให้นักวิจัยสามารถเทรนโมเดลขนาดใหญ่ได้ฟรี เราจึงมีโมเดลที่สามารถสร้างข้อความในแบบที่เราต้องการให้เลือกใช้ได้อย่างหลากหลายใน [HuggingFace Hub](https://huggingface.co/models)

เทคนิคการสร้างข้อความปลอมเพื่อมาเทรนโมเดลนี้เคยถูกใช้สำหรับเทรน[โมเดลแปลภาษาไทย-อังกฤษของ VISTEC](https://airesearch.in.th/releases/machine-translation-models/) ที่สามารถทำผลงานได้ดีที่สุด ณ วันตีพิมพ์ (หาอ่านงานวิจัยได้ที่ [scb-mt-en-th-2020: A Large English-Thai Parallel Corpus](https://arxiv.org/abs/2007.03541)) โดยเราใช้โมเดล [CTRL](https://github.com/salesforce/ctrl) สร้างข้อความรีวิวสินค้าระหว่าง 1-5 ดาวเป็นภาษาอังกฤษออกมาเพื่อให้นักแปลนำไปแปลเพื่อสร้างคู่ประโยคสำหรับเทรนโมเดลอีกที (ดูรายละเอียดได้ที่ [vistec-ai/fake_reviews](https://github.com/vistec-AI/fake_reviews))

แน่นอนว่าทุกวันนี้มีโมเดลที่สามารถสร้างข้อความได้แทบทุกรูปแบบ เราจะยกตัวอย่างโมเดล [GPT](https://arxiv.org/abs/2005.14165) สำหรับการสร้างรีวิวสินค้า 1 ดาวและ 5 ดาว

In [2]:
!pip install --upgrade keras

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 9.4 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.2.1 which is incompatible.


In [1]:
from transformers import pipeline
import pprint

#โหลด generator สำหรับรีวิว 1 ดาว
generator1 = pipeline(
    "text-generation",
    model="ehdwns1516/gpt2_review_star1",
)

RuntimeError: Failed to import transformers.pipelines because of the following error (look up to see its traceback):
module 'keras._tf_keras.keras' has no attribute '__internal__'

In [ ]:
context = '<|endoftext|>' #โมเดลต้องการ token พิเศษนี้นำหน้าเพื่อสร้างข้อความ
pprint.pprint([i['generated_text'] for i in generator1(context,
                                                      max_length = 60, #ให้แต่ละประโยคมีความยาว 60 token
                                                      num_return_sequences=3 #สร้างมา 3 ประโยค
                                                       )])

In [ ]:
#โหลด generator สำหรับรีวิว 1 ดาว
generator5 = pipeline(
    "text-generation",
    model="ehdwns1516/gpt2_review_star5",
)

In [ ]:
context = '<|endoftext|>' #โมเดลต้องการ token พิเศษนี้นำหน้าเพื่อสร้างข้อความ
pprint.pprint([i['generated_text'] for i in generator5(context,
                                                      max_length = 60, #ให้แต่ละประโยคมีความยาว 60 token
                                                      num_return_sequences=3 #สร้างมา 3 ประโยค
                                                       )])

### แปลชุดข้อมูลภาษาหลัก

อีกทางหนึ่งสำหรับการหาข้อมูลข้อความ โดยเฉพาะเป็นภาษาไทย คือคุณสามารถหาชุดข้อมูลภาษาหลัก เช่น อังกฤษหรือจีน มาแปลเป็นไทยได้ด้วยโมเดล open source เช่น `pythainlp.translate`

In [ ]:
!pip install -q accelerate==0.28.0

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import pprint

device = "cuda" # the device to load the model onto

# use bfloat16 to ensure the best performance.
model = AutoModelForCausalLM.from_pretrained("SeaLLMs/SeaLLM-7B-v2", torch_dtype=torch.bfloat16, device_map=device)
tokenizer = AutoTokenizer.from_pretrained("SeaLLMs/SeaLLM-7B-v2")

In [ ]:
messages = [
    {"role": "system", "content": "You are an elite English-Thai translator."},
    {"role": "user", "content": "Translate this sentence to English: I like to eat tom yum kung."}
]

encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt", add_generation_prompt=True)

model_inputs = encodeds.to(device)
model.to(device)

generated_ids = model.generate(model_inputs, max_new_tokens=1000, do_sample=True, pad_token_id=tokenizer.pad_token_id)
decoded = tokenizer.batch_decode(generated_ids)
pprint.pprint(decoded[0])

In [ ]:
messages = [
    {"role": "system", "content": "You are an elite English-Thai translator."},
    {"role": "user", "content": "แปลประโยคนี้เป็นอังกฤษให้หน่อย: โปแลนด์เป็นประเทศที่โด่งดังเรื่องดนตรีคลาสสิค"}
]

encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt", add_generation_prompt=True)

model_inputs = encodeds.to(device)
model.to(device)

generated_ids = model.generate(model_inputs, max_new_tokens=1000, do_sample=True, pad_token_id=tokenizer.pad_token_id)
decoded = tokenizer.batch_decode(generated_ids)
pprint.pprint(decoded[0])

### สร้าง Images จาก ML Models

เช่นเดียวกับข้อมูลข้อความ โมเดลขนาดใหญ่ที่ถูกเทรนด้วยรูปภาพมหาศาลก็สามารถสร้างรูปภาพปลอมเพื่อให้คุณไปเทรนโมเดลได้เช่นกัน เช่น [DALL-E Mini](https://github.com/borisdayma/dalle-mini) ที่เป็นโมเดล open source สร้างเลียนแบบโมเดล [DALL-E](https://openai.com/blog/dall-e/) ของ [OpenAI](https://openai.com/)

วิธีการสร้างรูปภาพค่อนข้างซับซ้อนและกินเวลานาน แนะนำให้ไปลองเล่นกันที่ [demo](https://huggingface.co/spaces/flax-community/dalle-mini) หรือดูรายละเอียดที่ [github repository](https://github.com/borisdayma/dalle-mini)

<img src=https://github.com/ai-builders/curriculum/raw/main/images/dalle_e_toast.png width="700px">


อีกตัวอย่างใกล้ตัวคือการใช้ Generative Adversarial Networks (GAN) เรียนรู้จากรูปที่เรามีอยู่เพื่อสร้างตัวอย่างเพิ่มขึ้น เช่น การสร้าง Anime faces จากรูปที่มีอยู่โดย [@konkuad](https://github.com/konkuad) ใน [konkuad/GANime](https://github.com/konkuad/GANime)

In [ ]:
!rm -r GANime
!git clone https://github.com/konkuad/GANime
!cd GANime; pip install .

In [ ]:
import torch
from GANime.gan import GAN

#initialize new model with random weights
seed_size=128
new_gan_model = GAN(seed_size)
#load model trained for 100 epochs
new_gan_model.gen.load_state_dict(torch.load('GANime/example_models/gen.pt'))
new_gan_model.dis.load_state_dict(torch.load('GANime/example_models/dis.pt'))
#infer generation
generated_images = new_gan_model.generate(num_rows=8,
               num_cols=8,
               plot=True,
               device='cpu',
               return_noise=False
              )

# Checkpoint ท้ายบท

## คำถามชวนคิดเกี่ยวกับบทเรียน

1. คุณคิดว่าปัญหาที่คุณกำลังจะแก้ด้วย ML นั้นปริมาณและคุณภาพของข้อมูลที่ใช้เทรน (Data-Centric AI) หรือประสิทธิภาพของโมเดล (Model-Centric AI) สำคัญกว่ากัน เพราะอะไร

2. คุณคิดว่า open data และ open source มีข้อดี-ข้อเสียอย่างไรต่อ 1) ผู้สร้างผลงาน 2) ผู้นำผลงานไปใช้ 3) ชุมชน 4) สังคมโดยรวม

3. คุณคิดว่าการใช้ข้อมูลสาธารณะจากเว็บไซต์ต่างๆมาเทรนโมเดลมีข้อควรระวังอะไรบ้าง

4. การเทรนโมเดลด้วยข้อมูลที่สร้างขึ้นมาจากอีกโมเดลหนึ่ง ("ข้อมูลปลอม") นั้นมีข้อดี-ข้อเสียอย่างไรบ้าง

## สิ่งที่ควรเตรียมพร้อมสำหรับทำโครงงาน

### ☑️ ทบทวนระบบการให้คะแนนโครงงานให้เรียบร้อย

AI Builders จะออกใบประกาศนียบัตรจบการศึกษาให้กับผู้เข้าร่วมโครงการที่ส่งโครงงานได้คะแนนอย่างน้อย 70 จาก 100 คะแนนตามเกณฑ์ต่อไปนี้เท่านั้น

1. problem statement; เหตุผลในการแก้ปัญหาเชิงธุรกิจ/ชีวิตประจำวันด้วย machine learning - 15 คะแนน
2. metrics and baselines; การให้เหตุผลเชื่อมโยงการแก้ปัญหากับตัวชี้วัดที่เลือก / การวัดผลเทียบกับวิธีแก้ปัญหาในปัจจุบัน - 15 คะแนน

**วันนี้เราจะคิดถึง 2 ข้อนี้เป็นพิเศษ**

**3. data collection and cleaning; การเก็บและทำความสะอาดข้อมูล - 15 คะแนน**


**4. exploratory data analysis; การทำความเข้าใจข้อมูล - 20 คะแนน**


5. modeling, validation and error analysis; การทำโมเดล, ทดสอบโมเดล และวิเคราะห์ข้อผิดพลาดของโมเดล - 20 คะแนน
6. deployment; การนำโมเดลไปใช้แก้ปัญหาจริง - 15 คะแนน

### ☑️ หาข้อมูลมาจากไหนดี

จากแหล่งข้อมูลทั้งหมดที่กล่าวมาในบทเรียนนี้ คุณคิดว่าจะหาชุดข้อมูลจากไหนมาเทรนโมเดล

### ☑️ คุณภาพของข้อมูล Labels และ Inputs

ข้อมูลที่คุณได้มาจากอินเตอร์เน็ตไม่ว่าจะดาวน์โหลดชุดข้อมูลของคนอื่น, scrape เว็บไซต์, ใช้โมเดลคนอื่นสร้างขึ้นมา คุณต้องทำให้แน่ใจว่าคุณภาพมันพร้อมใช้งานทั้ง Labels และ Inputs เช่น ถ้าอยากทำโมเดลแปลภาษาไทย-จีนก็ต้องทำให้แน่ใจว่าคู่ประโยคที่เราหามามันแปลตรงกันจริงๆ เป็นต้น

### ☑️ Train-validation-test Splits สมเหตุสมผลไหม

ใช้สามัญสำนึกและตรรกะในการแบ่งข้อมูลเป็น train, validation, test sets ทำให้มั่นใจว่าไม่มีตัวอย่างเดียวกันหลุดไปใน set อื่น เช่น

* ในการทำ face recognition ไม่ควรมีรูปคนๆเดียวกันในมากกว่า 1 set

* หากทำ time series forecasting เรียงข้อมูลจากเก่าสุดไปใหม่สุดใน `train`, `validation` และ `test` เพื่อไม่ให้โมเดลเห็นข้อมูลจากอนาคต

* หากทำ product recommendation ไม่ควรมีข้อมูลของลูกค้าคนเดียวกันในมากกว่า 1 set เพราะจะทำให้เราเดาว่าเขาเป็นลูกค้าประเภทไหนได้ด้วยข้อมูลที่ไม่ควรเห็น ฯลฯ

ทั้งนี้ขึ้นอยู่กับวิจารณญาณของผู้สร้างโมเดล เรียนรู้เพิ่มเติมเกี่ยวกับการ split ข้อมูลได้จาก เรียนรู้เพิ่มเติมเกี่ยวกับ metric ได้จาก [DS&AI Academy](https://www.youtube.com/playlist?list=PL1kutgc5YvC841TmatsLuj4sDKkwj95Dz)